## CLEANING

### Notebook structure

[1. Checking for sources on participant demographic information and diagnosis](#demog)   
- rectifying missing values in columns gender, race, ethnicity, marital status, age, ...
- assessing information sources

[2. Neuropsychological tests - cleaning](#tests)  
- removing irrelevant columns and preparing each test dataset for exploration and visualization in the notebook EDA.ipynb


In [1]:
# Importing libraries.

import pandas as pd
import numpy as np
import math

import datetime as dt
from datetime import date, timedelta

import plotly 
import plotly.express as px
import seaborn as sns

from pandas.api.types import CategoricalDtype
from sklearn import preprocessing

import glob

# from scipy.cluster.hierarchy import fastcluster
# import missingno # to identify and visualize missing data prior to ML

### <a id="demog">1. Checking for sources on participant demographic information and diagnosis</a>    

There are 5 neuropsychological tests that we want to anlayze:    
- **Clinical Dementia Rating (CDR)**   
- **Geriatric Depression Scale (GDS)**   
- **Mini Mental State Examination (MMSE)**    
- **Neuropsychological Battery Tests (NEUROBAT)**    
- **Montreal Cognitive Assessment (MoCA)**    

The test datasets contain the participants' scores and **no** demographic information.

Ideally, we would like to investigate the relationship between the test scores and other independent variables, such as participants' age, gender, ethnicity, etc. We would also like to check the correlation of several variables with the participant's diagnosis.

To achieve this, we want to examine the dataframe with the **demographic information**, so we can freely merge its columns with the test datasets as needed. 

The purpose is to obtain: **gender, year of birth, marital status, education, ethnicity, race, diagnosis**, and other relevant information.

Since the ADNI database has grown organically with each ADNI phase over the last 17 years, it exhibits a complex structure and the data is scattered over several tables. To gain some insight, we will need to analyze different sources to obtain the best quality data:
- PTDEMOG.csv (*demographics*)
- REGISTRY.csv (*phases, exam dates and codes*)
- ADNIMERGE.csv (*selected data over all ADNI phases: ADNI1, ADNI2, ADNIGO, ADNI3 merged*) 
- DXSUM_PDXCONV_ADNIALL.csv (*diagnostic information*)

We will inspect these files in advance so that when we merge selected columns to test tables we will know for sure that resulting NaN values cannot be obtained or inferred.


#### Demographics (gender, education, work history, ethnicity, etc.)

In [2]:
# PTDEMOG data dictionary.
demog_dict = pd.read_csv('../../data/tabular_data/PTDEMOG_dict.csv')
demog_dict.style.set_properties(**{'text-align': 'left'}).hide_index()
# demog_dict

ID,"""Record ID"""
RID,"""Participant roster ID"""
SITEID,"""Site ID"""
VISCODE,"""Visit code"""
VISCODE2,"""Translated visit code: longutudinal progression"""
USERDATE,"""Date record created"""
USERDATE2,"""Date record last updated"""
PTSOURCE,"""Information Source: 1=Participant Visit;2=Telephone Call"""
PTGENDER,"""Participant Gender: 1=Male; 2=Female"""
PTDOBMM,"""2a. Participant Month of Birth; 1..12"""
PTDOBYY,"""2b. Participant Year of Birth; 1900..1980"""


#### Examining the demographic data

In [3]:
# Cohort demography, partial: PTDEMOG.csv
df = pd.read_csv('../../data/tabular_data/PTDEMOG.csv')
df.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSOURCE,PTGENDER,...,PTTLANG,PTPLANG,PTPSPEC,PTCOGBEG,PTMCIBEG,PTADBEG,PTADDX,PTETHCAT,PTRACCAT,update_stamp
0,ADNI1,18,2,107,sc,sc,2005-08-17,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,-4.0,NaN,2.0,5.0,2005-08-17 00:00:00.0
1,ADNI1,20,1,10,f,f,2005-08-18,NaN,1,2.0,...,-4.0,-4.0,-4,NaN,NaN,-4.0,NaN,-4.0,-4.0,2005-08-18 00:00:00.0
2,ADNI1,22,3,107,sc,sc,2005-08-18,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,1999.0,NaN,2.0,5.0,2005-08-18 00:00:00.0
3,ADNI1,24,4,10,sc,sc,2005-08-18,NaN,1,1.0,...,2.0,2.0,-4,NaN,NaN,-4.0,NaN,1.0,5.0,2005-08-18 00:00:00.0
4,ADNI1,26,5,107,sc,sc,2005-08-23,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,-4.0,NaN,2.0,5.0,2005-08-23 00:00:00.0


In [4]:
df.shape

(4716, 32)

In [5]:
# No. of entries to start with.
dem_start_entries = df.shape[0]

In [6]:
df.isnull().sum()

Phase              0
ID                 0
RID                0
SITEID             0
VISCODE            0
VISCODE2           3
USERDATE           0
USERDATE2       3158
PTSOURCE           0
PTGENDER         119
PTDOBMM          176
PTDOBYY          176
PTHAND           132
PTMARRY            4
PTEDUCAT         132
PTWORKHS        2318
PTWORK            13
PTWRECNT          24
PTNOTRT           18
PTRTYR           264
PTHOME             8
PTOTHOME         950
PTTLANG          119
PTPLANG          119
PTPSPEC          945
PTCOGBEG        2167
PTMCIBEG        4267
PTADBEG         3204
PTADDX          2181
PTETHCAT         119
PTRACCAT         119
update_stamp       0
dtype: int64

In [7]:
# How mant valied entries do we have about the PARTICIPANT's DATE OF DIAGNOSIS?
df['PTADDX'].unique()

array([  nan, 9999., 2007., 2009., 2008., 2010., 2006., 2004., 2011.,
       2005., 1986., 2012., 1999., 1987., 2003., 2017., 2016., 2013.,
       2015., 2014., 1985., 2018., 2019., 2020., 2021.])

In [8]:
# Hidden missing values as 9999:
df['PTADDX'] = df['PTADDX'].replace(9999, np.NaN)

In [9]:
df['PTADDX'].isna().sum()

4308

We have date of AD diagnosis for only about 10% of the participants, it would be hard to draw a valid conclusion on when people in our cohort got sick.

In [10]:
# Beginning of mild cog. impairments, symptoms year.
df['PTMCIBEG'].unique()

array([  nan, 2006., 2005., 2009., 2008., 2007., 1997., 2000., 1996.,
       1999., 1985., 2010., 1993., 2004., 1998., 2002., 1994., 2001.,
       2003., 1995., 1986., 1991., 1990.])

In [11]:
df['PTMCIBEG'].isna().sum()

4267

We have information on cognitive imapirment onset for 449/4716 people, which is too small of a number to make any generalizations. As our focus is not so much on development of the disease and survival once you are diagnosed, we might remove these columns to obtain a cleaner dataframe.

In [12]:
# Beginning of AD symptoms, year.
df['PTADBEG'].unique()

array([-4.000e+00,  1.999e+03,  2.002e+03,  2.001e+03,  2.005e+03,
        2.000e+03,  2.003e+03,  1.998e+03,  2.004e+03,  1.997e+03,
        1.996e+03,  2.006e+03,  1.993e+03,  1.994e+03,  1.987e+03,
       -1.000e+00,  1.995e+03,  2.009e+03,  2.010e+03,        nan,
        2.008e+03,  1.985e+03,  2.007e+03,  1.991e+03])

In [13]:
# Drop unused columns.
drop_cols = ['USERDATE', 'USERDATE2', 'PTSOURCE', 'PTDOBMM', 'PTWORKHS', 
             'PTWRECNT', 'PTNOTRT', 'PTRTYR', 'PTWORK', 'PTOTHOME', 
             'PTTLANG', 'PTPSPEC', 'PTCOGBEG', 'PTMCIBEG', 'PTADBEG', 
             'PTADDX', 'update_stamp']

dem = df.drop(columns=drop_cols, axis=1)
# dem.head()

In [14]:
# Basic information overview.
print('Dataframe shape:', df.shape)
print()
dem.info()

Dataframe shape: (4716, 32)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4716 entries, 0 to 4715
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Phase     4716 non-null   object 
 1   ID        4716 non-null   int64  
 2   RID       4716 non-null   int64  
 3   SITEID    4716 non-null   int64  
 4   VISCODE   4716 non-null   object 
 5   VISCODE2  4713 non-null   object 
 6   PTGENDER  4597 non-null   float64
 7   PTDOBYY   4540 non-null   float64
 8   PTHAND    4584 non-null   float64
 9   PTMARRY   4712 non-null   float64
 10  PTEDUCAT  4584 non-null   float64
 11  PTHOME    4708 non-null   float64
 12  PTPLANG   4597 non-null   float64
 13  PTETHCAT  4597 non-null   float64
 14  PTRACCAT  4597 non-null   float64
dtypes: float64(9), int64(3), object(3)
memory usage: 552.8+ KB


We know, that in generate the missing values in ADNI datasets are encoded also with -1 and -4, so we'll change these values to NaN.

In [15]:
dem = dem.replace([-1], 'NaN')
dem = dem.replace([-4], 'NaN')
dem = dem.replace(['-1'], 'NaN')
dem = dem.replace(['-4'], 'NaN')

# Missing VISCODE2
# VISCODE2 = translated VISCODE into month visit for each participant
mv = dem[dem['VISCODE2'].isnull()]
mv

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTHOME,PTPLANG,PTETHCAT,PTRACCAT
4514,ADNI3,90663,6214,42,sc,NaN,2.0,1952.0,1.0,1.0,18.0,1.0,1.0,2.0,4.0
4596,ADNI3,109968,6617,10,sc,NaN,2.0,1943.0,NaN,NaN,NaN,1.0,1.0,3.0,7.0
4714,ADNI3,138453,6949,12,sc,NaN,2.0,1951.0,1.0,1.0,14.0,2.0,2.0,1.0,5.0


In [16]:
dem['VISCODE2'] =  dem['VISCODE2'].fillna('sc')

In [17]:
dem.isnull().sum()

Phase         0
ID            0
RID           0
SITEID        0
VISCODE       0
VISCODE2      0
PTGENDER    119
PTDOBYY     176
PTHAND      132
PTMARRY       4
PTEDUCAT    132
PTHOME        8
PTPLANG     119
PTETHCAT    119
PTRACCAT    119
dtype: int64

In [18]:
# Encode unknown values in certain columns, as per data dictionary. Eg. Unknown marital status is coded with '5'.
dem['PTETHCAT'] = dem['PTETHCAT'].fillna(3) 
dem['PTRACCAT'] = dem['PTRACCAT'].fillna(7) 
dem['PTMARRY'] = dem['PTMARRY'].fillna(5)

119 missing values in 4 columns, as well as exactly 132 missings in two columns seems to be a strange coincidence, so we will check, what is with that.       

But first, let's check for duplicates.

In [19]:
# How many unique individuals? RID is personal ID number. Possible duplicates can be detected.
dem['RID'].nunique()

3940

There are 4716 entries, but only 3940  study participants. Let's check, which participants are entered several times and try to understand why and exclude the possibility of repeated entries.

In [20]:
# Count of each of the distinct values of RID column
repeated = dem.RID.value_counts().sort_values(ascending=False).reset_index()
# rep_RID = repeated.loc[repeated[repeated > 1]].tolist()
repeated = repeated[repeated['RID'] > 1]
repeated.rename(columns = {'index':'RID', 'RID':'count'}, inplace = True)
rids = repeated['RID'].tolist()
# rids

In [21]:
# Check the rows with repeated RIDs
rep_rid = dem[dem.RID.isin(rids)]
rep_rid = rep_rid.sort_values(by = 'RID')
rep_rid.head(12)

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTHOME,PTPLANG,PTETHCAT,PTRACCAT
0,ADNI1,18,2,107,sc,sc,1.0,1931.0,2.0,1.0,16.0,1.0,1.0,2.0,5.0
1571,ADNIGO,304,2,8,sc,sc,1.0,1931.0,2.0,2.0,16.0,1.0,1.0,2.0,5.0
2404,ADNI2,636,2,8,v06,m72,1.0,1931.0,2.0,2.0,16.0,1.0,1.0,2.0,5.0
7,ADNI1,32,8,107,sc,sc,2.0,1921.0,2.0,2.0,18.0,1.0,1.0,2.0,5.0
1579,ADNIGO,322,8,8,sc,sc,2.0,1921.0,2.0,2.0,18.0,1.0,1.0,2.0,5.0
2430,ADNI2,688,8,8,v06,m72,2.0,1921.0,2.0,2.0,18.0,1.0,1.0,2.0,5.0
13,ADNI1,46,15,4,sc,sc,1.0,1924.0,1.0,1.0,18.0,1.0,1.0,2.0,5.0
2541,ADNI2,910,15,40,v06,m72,1.0,1924.0,1.0,1.0,18.0,1.0,1.0,2.0,5.0
22,ADNI1,64,21,107,sc,sc,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,2.0,4.0
1608,ADNIGO,382,21,8,sc,sc,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,2.0,4.0


There are no duplicates, it appears we're dealing with participants, who continued through several ADNI phases, so the entries are valid.    

Coincidentally, we also partially solve the the missing values problem (example: participant RID = 21): gender and other columns are not missing at all, they were just not entered for all ADNI phases. Do participants with missing values have these entered at screening?

The ethnic category (PTETHCAT), for example, is important as a 'boolean' value, meaning, if a participant is Hispanic or not. For classification models this is informative, because in the literature we've learned that Hispanic people tend to have a higer risk to get Alzheimer's and they get sick at a younger age.

In [22]:
# Missing values at screening exam.
scr_values = dem[dem.VISCODE.isin(['sc'])]
scr_values.isnull().sum()

Phase        0
ID           0
RID          0
SITEID       0
VISCODE      0
VISCODE2     0
PTGENDER     0
PTDOBYY      0
PTHAND      13
PTMARRY      0
PTEDUCAT     8
PTHOME       8
PTPLANG      0
PTETHCAT     0
PTRACCAT     0
dtype: int64

Apart from couple of missing values on handedness etc., the df seems to have the values on gender, ethnicity and other more relevant variables, so we will tolerate the few missing values  in 3 columns and handle them if we need the columns later.

In [23]:
# Using values from other rows belonging to the same participant RID to rectify missing gender, education, marital status, etc.

# Use fillna and pass the df without NaN rows, setting the index to ['RID'] and then calling map which will perform a lookup.
# RID is not unique, so drop duplicates.

dem['PTGENDER'] = dem['PTGENDER'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTGENDER']))
dem['PTDOBYY'] = dem['PTDOBYY'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTDOBYY']))
dem['PTHAND'] = dem['PTHAND'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTHAND']))
dem['PTMARRY'] = dem['PTMARRY'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTMARRY']))
dem['PTEDUCAT'] = dem['PTEDUCAT'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTEDUCAT']))
dem['PTPLANG'] = dem['PTPLANG'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTPLANG']))
dem['PTETHCAT'] = dem['PTETHCAT'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTETHCAT']))
dem['PTRACCAT'] = dem['PTRACCAT'].fillna(dem['RID'].map(dem.dropna().drop_duplicates('RID').set_index('RID')['PTRACCAT']))

In [24]:
dem.isnull().sum()

Phase        0
ID           0
RID          0
SITEID       0
VISCODE      0
VISCODE2     0
PTGENDER     0
PTDOBYY      0
PTHAND      13
PTMARRY      0
PTEDUCAT    10
PTHOME       8
PTPLANG      0
PTETHCAT     0
PTRACCAT     0
dtype: int64

In [25]:
# Unknown handiness, encode with 0.
dem['PTHAND'] = dem['PTHAND'].fillna(0)

In [26]:
dem_na = dem[dem.isna().any(axis=1)]
dem_na

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTHOME,PTPLANG,PTETHCAT,PTRACCAT
2986,ADNI2,1800,4667,4,v01,sc,2.0,1944.0,1.0,1.0,NaN,1.0,1.0,2.0,5.0
3726,ADNI2,3284,4273,50,v01,sc,1.0,1947.0,1.0,1.0,NaN,8.0,3.0,3.0,7.0
3874,ADNI3,21349,6123,49,sc,sc,1.0,1937.0,0,1.0,18.0,NaN,1.0,2.0,5.0
3943,ADNI3,29098,6201,49,sc,sc,1.0,1939.0,0,1.0,12.0,NaN,1.0,3.0,4.0
4242,ADNI3,54264,6453,44,sc,sc,1.0,1960.0,0,1.0,NaN,NaN,1.0,2.0,5.0
4268,ADNI3,56372,6435,30,sc,sc,2.0,1953.0,1.0,1.0,NaN,1.0,1.0,2.0,5.0
4319,ADNI3,63501,6587,57,sc,sc,1.0,1936.0,1.0,1.0,12.0,NaN,1.0,2.0,5.0
4430,ADNI3,79361,6608,57,sc,sc,1.0,1949.0,0,5.0,NaN,NaN,1.0,2.0,7.0
4461,ADNI3,84367,6387,50,sc,sc,2.0,1954.0,0,5,NaN,NaN,1.0,2.0,5.0
4462,ADNI3,84346,6553,50,sc,sc,2.0,1958.0,0,5,NaN,NaN,1.0,2.0,5.0


In [27]:
dem[dem['RID'] == 4667]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTHOME,PTPLANG,PTETHCAT,PTRACCAT
2986,ADNI2,1800,4667,4,v01,sc,2.0,1944.0,1.0,1.0,NaN,1.0,1.0,2.0,5.0


In [28]:
dem.isnull().sum()

Phase        0
ID           0
RID          0
SITEID       0
VISCODE      0
VISCODE2     0
PTGENDER     0
PTDOBYY      0
PTHAND       0
PTMARRY      0
PTEDUCAT    10
PTHOME       8
PTPLANG      0
PTETHCAT     0
PTRACCAT     0
dtype: int64

We still have a couple of missing values, but mostly with participants who have completed only one visit  in 2. or 3. Phase. (We've checked the entries individually with "dem[dem['RID'] == <RID>]"). These participants are probably missing entries in test results, since not many tests were performed in first visit and they can be removed.

Now we will try to understand the visit codes, if we can use them to group, merge and filter entries.

In [29]:
vis_dict = pd.read_csv('../../data/tabular_data/VISITS.csv')
vis_dict

,Phase,ID,VISCODE,VISNAME,VISORDER
0,ADNI1,1,sc,Screening,1.0
1,ADNI1,2,bl,Baseline,2.0
2,ADNI1,3,m06,Month 6,3.0
3,ADNI1,4,m12,Month 12,4.0
4,ADNI1,5,m18,Month 18,5.0
5,ADNI1,6,m24,Month 24,6.0
6,ADNI1,7,m30,Month 30,7.0
7,ADNI1,8,m36,Month 36,8.0
8,ADNI1,9,m42,Month 42,9.0
9,ADNI1,10,m48,Month 48,10.0


In [30]:
vis2 = pd.read_csv('../../data/tabular_data/ADNI2_VISITID.csv')
# vis2

VISCODE2 appears to be longitudinal information about the study progress in months (VISCODE expressed in months).    
However, some discrepancy appears between VISCODE pre- and post- ADNI2 Phase, so we might need to order the progression by 'Phase', 'RID' and 'VISCODE' later.   

Let's save clean processed dem df to new csv.

In [31]:
dem = dem.dropna()

In [32]:
# How many entries did we lose to cleaning?
dem_finish_entries = dem.shape[0]
dropped_entries = dem_start_entries - dem_finish_entries
print('Dropped entries:', dropped_entries)
print('Which is', np.round(dropped_entries/dem_start_entries*100, 2), '% of entries removed.')

Dropped entries: 13
Which is 0.28 % of entries removed.


In [33]:
# Writing dem to csv.
df = dem
df.to_csv('../../data/tabular_data/clean_PTDEMOG.csv', index=False)

#### ADNIMERGE - selected merged datatables with data from ADNI1, ADNI2, ADNIGO, ADNI3

Next, we will check the file with data over all ADNI phases.

In [34]:
# ADNIMERGE dictionary.
df = pd.read_csv('../../data/tabular_data/ADNIMERGE_dict.csv')

# changing the max_rows value
pd.set_option("display.max_rows", 115)

adnimerge_dict = df[['FLDNAME', 'TEXT']]
adnimerge_dict

,FLDNAME,TEXT
0,RID,Participant roster ID
1,PTID,Original study protocol
2,ABETA,CSF ABETA
3,VISCODE,Visit code
4,SITE,Site
5,COLPROT,Study protocol of data collection
6,ORIGPROT,Original study protocol
7,EXAMDATE,Date
8,DX_bl,Baseline Dx
9,AGE,Age


In [35]:
# All ADNI phases, merged data, complementing PTDEMOG.csv: ADNIMERGE.csv
df = pd.read_csv('../../data/tabular_data/ADNIMERGE.csv')
df.head()

C:\Users\kfatur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,...,NaN,1.25173,NaN,NaN,NaN,0.000000,0.00000,0,0,2022-05-14 05:01:08.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,...,22.83,1.22163,NaN,NaN,NaN,0.000000,0.00000,0,0,2022-05-14 05:01:08.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,...,22.83,1.22163,NaN,NaN,NaN,0.498289,5.96721,6,6,2022-05-14 05:01:08.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,...,22.83,1.22163,NaN,NaN,NaN,0.999316,11.96720,12,12,2022-05-14 05:01:08.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,...,22.83,1.22163,NaN,NaN,NaN,1.998630,23.93440,24,24,2022-05-14 05:01:08.0


In [36]:
df.columns.tolist()

['RID',
 'COLPROT',
 'ORIGPROT',
 'PTID',
 'SITE',
 'VISCODE',
 'EXAMDATE',
 'DX_bl',
 'AGE',
 'PTGENDER',
 'PTEDUCAT',
 'PTETHCAT',
 'PTRACCAT',
 'PTMARRY',
 'APOE4',
 'FDG',
 'PIB',
 'AV45',
 'FBB',
 'ABETA',
 'TAU',
 'PTAU',
 'CDRSB',
 'ADAS11',
 'ADAS13',
 'ADASQ4',
 'MMSE',
 'RAVLT_immediate',
 'RAVLT_learning',
 'RAVLT_forgetting',
 'RAVLT_perc_forgetting',
 'LDELTOTAL',
 'DIGITSCOR',
 'TRABSCOR',
 'FAQ',
 'MOCA',
 'EcogPtMem',
 'EcogPtLang',
 'EcogPtVisspat',
 'EcogPtPlan',
 'EcogPtOrgan',
 'EcogPtDivatt',
 'EcogPtTotal',
 'EcogSPMem',
 'EcogSPLang',
 'EcogSPVisspat',
 'EcogSPPlan',
 'EcogSPOrgan',
 'EcogSPDivatt',
 'EcogSPTotal',
 'FLDSTRENG',
 'FSVERSION',
 'IMAGEUID',
 'Ventricles',
 'Hippocampus',
 'WholeBrain',
 'Entorhinal',
 'Fusiform',
 'MidTemp',
 'ICV',
 'DX',
 'mPACCdigit',
 'mPACCtrailsB',
 'EXAMDATE_bl',
 'CDRSB_bl',
 'ADAS11_bl',
 'ADAS13_bl',
 'ADASQ4_bl',
 'MMSE_bl',
 'RAVLT_immediate_bl',
 'RAVLT_learning_bl',
 'RAVLT_forgetting_bl',
 'RAVLT_perc_forgetting_bl',

In [37]:
# 'ORIGPROT' = Original study protocol, which is the closest information to indicate Phase
am = df[['RID', 'ORIGPROT', 'SITE', 'VISCODE', 'EXAMDATE',
         'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT',
         'PTRACCAT', 'PTMARRY', 'DX', 'EXAMDATE_bl'
        ]] 
am.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15911 entries, 0 to 15910
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   RID          15911 non-null  int64  
 1   ORIGPROT     15911 non-null  object 
 2   SITE         15911 non-null  int64  
 3   VISCODE      15911 non-null  object 
 4   EXAMDATE     15911 non-null  object 
 5   DX_bl        15883 non-null  object 
 6   AGE          15903 non-null  float64
 7   PTGENDER     15911 non-null  object 
 8   PTEDUCAT     15911 non-null  int64  
 9   PTETHCAT     15911 non-null  object 
 10  PTRACCAT     15911 non-null  object 
 11  PTMARRY      15910 non-null  object 
 12  DX           11120 non-null  object 
 13  EXAMDATE_bl  15911 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 1.7+ MB


In [38]:
am.rename(columns = {'ORIGPROT':'Phase'})

,RID,Phase,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,DX,EXAMDATE_bl
0,2,ADNI1,11,bl,2005-09-08,CN,74.3,Male,16,Not Hisp/Latino,White,Married,CN,2005-09-08
1,3,ADNI1,11,bl,2005-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,Dementia,2005-09-12
2,3,ADNI1,11,m06,2006-03-13,AD,81.3,Male,18,Not Hisp/Latino,White,Married,Dementia,2005-09-12
3,3,ADNI1,11,m12,2006-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,Dementia,2005-09-12
4,3,ADNI1,11,m24,2007-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,Dementia,2005-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15906,5154,ADNI2,3,m90,2021-02-09,SMC,72.4,Female,18,Not Hisp/Latino,White,Never married,NaN,2013-06-10
15907,6341,ADNI3,7,m48,2022-05-09,LMCI,67.1,Male,12,Not Hisp/Latino,White,Married,NaN,2018-05-29
15908,6362,ADNI3,98,m36,2021-05-11,SMC,82.9,Female,18,Not Hisp/Latino,Asian,Widowed,NaN,2018-06-04
15909,7039,ADNI3,16,bl,2022-03-10,AD,61.6,Female,18,Not Hisp/Latino,Black,Married,Dementia,2022-03-10


In [39]:
# Add approximate year of birth column to the 'am' dataframe.
# Is the date of appropriate type?
am.isnull().sum()

RID               0
ORIGPROT          0
SITE              0
VISCODE           0
EXAMDATE          0
DX_bl            28
AGE               8
PTGENDER          0
PTEDUCAT          0
PTETHCAT          0
PTRACCAT          0
PTMARRY           1
DX             4791
EXAMDATE_bl       0
dtype: int64

In [40]:
am['PTMARRY'].unique()

array(['Married', 'Divorced', 'Widowed', 'Never married', 'Unknown', nan],
      dtype=object)

In [41]:
am['PTMARRY'] = am['PTMARRY'].fillna('Unknown') 

C:\Users\kfatur\AppData\Local\Temp/ipykernel_33648/754673376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  am['PTMARRY'] = am['PTMARRY'].fillna('Unknown')


In [42]:
# Which entries are missing the diagnosis?
df = am.drop(['DX'], axis = 1)
am_na = df[df.isna().any(axis=1)]
print(am_na.shape)
# am_na

(36, 13)


In [43]:
# We would like to obtain YOB from the above PTDEMOG df, so we will check if the RIDs from the entries with missing values are in the dem df above.
rids = dem['RID'].tolist()
match_rid = am_na[am_na['RID'].isin(rids)]
print(match_rid.shape)
match_rid.sort_values(by = 'RID')

(31, 13)


,RID,ORIGPROT,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,EXAMDATE_bl
14997,6361,ADNI3,130,bl,2018-07-25,NaN,78.0,Female,15,Not Hisp/Latino,White,Widowed,2018-07-25
15038,6390,ADNI3,130,bl,2018-08-06,NaN,68.9,Female,15,Not Hisp/Latino,White,Married,2018-08-06
15154,6466,ADNI3,36,bl,2018-07-17,NaN,68.1,Female,14,Not Hisp/Latino,White,Divorced,2018-07-17
15212,6505,ADNI3,29,bl,2018-09-14,CN,NaN,Female,20,Hisp/Latino,White,Married,2018-09-14
15351,6598,ADNI3,53,m30,2021-05-03,LMCI,NaN,Male,12,Not Hisp/Latino,White,Married,2018-10-22
15350,6598,ADNI3,53,m12,2019-10-29,LMCI,NaN,Male,12,Not Hisp/Latino,White,Married,2018-10-22
15349,6598,ADNI3,53,bl,2018-10-22,LMCI,NaN,Male,12,Not Hisp/Latino,White,Married,2018-10-22
15384,6627,ADNI3,37,bl,2019-02-27,NaN,67.7,Female,14,Not Hisp/Latino,White,Divorced,2019-02-27
15440,6701,ADNI3,32,bl,2019-08-22,NaN,71.1,Female,14,Not Hisp/Latino,Black,Married,2019-08-22
12957,6712,ADNI3,19,bl,2019-04-24,NaN,90.6,Male,17,Not Hisp/Latino,White,Married,2019-04-24


For 31 people we might get missing values from the dem df. Diagnosis will be takend form the file DXSUM_PDXCONV_ADNIALL.csv, so here we drop the column. Age will could be calculated based on YOB from dem df.

In [44]:
am = am.drop(['DX_bl', 'DX'], axis=1)

In [45]:
# Can we obtain age by substracting year of birth dem['PTDOBYY'] from baseline exam date am['EXAMDATE_bl']?
# If values match for a subject who has age entered in am df, then hypothesis is valid.
# Compare with age from dem df for RID where the year of birth is known.

am['EXAMDATE'] = pd.to_datetime(am['EXAMDATE'])
am['EXAMDATE_bl'] = pd.to_datetime(am['EXAMDATE_bl'])

#Age from am df.
age1 = am[am['RID'] == 6466]['AGE'].reset_index(drop=True)
age1

# Year of birth from dem df.
dem_yob = dem[dem['RID'] == 6598]
yob = (dem_yob['PTDOBYY']).astype('int64').reset_index(drop=True).astype(float)
yob

# Exam date from am df.
exam_d = am[am['RID'] == 6598]
examyear = (exam_d['EXAMDATE_bl']).dt.year.reset_index(drop=True).iloc[0].astype(float)
examyear

# Age calculated with yob from dem and exam date from am.
age2 = (examyear - yob).reset_index(drop=True).astype(float)[0]
age2

print(age1)
print(age2)

0    68.1
Name: AGE, dtype: float64
71.0


With a 3-year difference between entered ad inferred age, it is not acceptable to calculate the age from the baseline exam date. Inconsistencies with age are, according to online discussions, a common problem with the ADNI data, since age info might have been collected at one date and exam conducted on another.   
To ensure the integrity of data, we will drop the rows and, where available use the data for these participants from dem df.

In [46]:
am = am.dropna(subset=['AGE'])

In [47]:
am.isnull().sum()

RID            0
ORIGPROT       0
SITE           0
VISCODE        0
EXAMDATE       0
AGE            0
PTGENDER       0
PTEDUCAT       0
PTETHCAT       0
PTRACCAT       0
PTMARRY        0
EXAMDATE_bl    0
dtype: int64

In [48]:
print(am.shape)

# Are there the same no. of participants as in PTDEMOG - 3940?
print(am['RID'].nunique())

(15903, 12)
2397


PTDEMOG.csv and ADNIMERGE.csv seem to have a different number of participants. We know that some people failed screening and some discontinued or were excluded. We will just note, that we migt need to combine these two sources to obtain complete demographic information on our study cohort.

In [49]:
# Writing am to csv. Comment/uncomment as needed
df = am
df.to_csv('../../data/tabular_data/clean_ADNIMERGE.csv', index=False)

#### Diagnostic information

Now let's study the diagnostic table, the hopes are high. Fingers crossed that we will get as many diagnostic entries as possible.

In [50]:
# Diagnosis: DXSUM_PDXCONV_ADNIALL.csv
df = pd.read_csv('../../data/tabular_data/DXSUM_PDXCONV_ADNIALL.csv')
diag = df[['Phase', 'RID', 'PTID', 'SITEID', 'VISCODE', 'VISCODE2','EXAMDATE',
         'DXCHANGE', 'DXCURREN', 'DIAGNOSIS'
        ]] 

diag.head()

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DXCURREN,DIAGNOSIS
0,ADNI1,2,011_S_0002,107,bl,bl,2005-09-29,NaN,1.0,NaN
1,ADNI1,2,011_S_0002,107,m06,m06,2006-03-06,NaN,1.0,NaN
2,ADNI1,3,011_S_0003,107,bl,bl,2005-09-30,NaN,3.0,NaN
3,ADNI1,3,011_S_0003,107,m06,m06,2006-03-13,NaN,3.0,NaN
4,ADNI1,3,011_S_0003,107,m12,m12,2006-09-13,NaN,3.0,NaN


The head does not look too promising.
Current diagnosis variable names change by the phase in the diagnosis summary table.
Quick recap:
- ADNI1: DXCURREN 1=NL; 2=MCI; 3=AD
- ADNIGO/2: DXCHANGE    
    1=Stable: NL to NL;    
    2=Stable: MCI to MCI;  
    3=Stable: Dementia to Dementia;   
    4=Conversion: NL to MCI;   
    5=Conversion: MCI to Dementia; 
    6=Conversion: NL to Dementia;  
    7=Reversion: MCI to NL;   
    8=Reversion: Dementia to MCI;   
    9=Reversion: Dementia to NL    
- ADNI3: DIAGNOSIS 1=CN; 2=MCI; 3=Dementia

In [51]:
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12996 entries, 0 to 12995
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Phase      12996 non-null  object 
 1   RID        12996 non-null  int64  
 2   PTID       12996 non-null  object 
 3   SITEID     12996 non-null  int64  
 4   VISCODE    12996 non-null  object 
 5   VISCODE2   12990 non-null  object 
 6   EXAMDATE   12966 non-null  object 
 7   DXCHANGE   6146 non-null   float64
 8   DXCURREN   3868 non-null   float64
 9   DIAGNOSIS  2971 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 1015.4+ KB


In [52]:
diag_start_entries = diag.shape[0]
diag_start_entries

12996

In [53]:
diag['EXAMDATE'] = pd.to_datetime(diag['EXAMDATE']).dt.date

C:\Users\kfatur\AppData\Local\Temp/ipykernel_33648/4249674630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diag['EXAMDATE'] = pd.to_datetime(diag['EXAMDATE']).dt.date


In [54]:
diag.isnull().sum()

Phase            0
RID              0
PTID             0
SITEID           0
VISCODE          0
VISCODE2         6
EXAMDATE        30
DXCHANGE      6850
DXCURREN      9128
DIAGNOSIS    10025
dtype: int64

Yikes! Diagnostic columns have missing values in the 1000s. 

In [55]:
missing = np.round((9128/diag.shape[0]*100), decimals=1)
missing
print(missing, "% of the values in the current diagnosis column ['DXCURREN'] are missing.")

70.2 % of the values in the current diagnosis column ['DXCURREN'] are missing.


In [56]:
# What is with the missing diagnosis in the diagnosis file?
df = diag[diag['DIAGNOSIS'].isnull()]
df

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DXCURREN,DIAGNOSIS
0,ADNI1,2,011_S_0002,107,bl,bl,2005-09-29,NaN,1.0,NaN
1,ADNI1,2,011_S_0002,107,m06,m06,2006-03-06,NaN,1.0,NaN
2,ADNI1,3,011_S_0003,107,bl,bl,2005-09-30,NaN,3.0,NaN
3,ADNI1,3,011_S_0003,107,m06,m06,2006-03-13,NaN,3.0,NaN
4,ADNI1,3,011_S_0003,107,m12,m12,2006-09-13,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...
11947,ADNI3,5273,135_S_5273,52,y2,NaN,NaT,NaN,NaN,NaN
12112,ADNI3,6701,032_S_6701,23,bl,bl,NaT,NaN,NaN,NaN
12507,ADNI3,6898,003_S_6898,2,sc,sc,NaT,NaN,NaN,NaN
12565,ADNI3,5158,029_S_5158,21,y2,m78,NaT,NaN,NaN,NaN


In [57]:
# Do some participants have at least diagnosis at some visits, but not on others?
diag_by_rid = diag.sort_values(['RID', 'EXAMDATE'])
diag_by_rid.head(100)

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DXCURREN,DIAGNOSIS
0,ADNI1,2,011_S_0002,107,bl,bl,2005-09-29,NaN,1.0,NaN
1,ADNI1,2,011_S_0002,107,m06,m06,2006-03-06,NaN,1.0,NaN
2134,ADNI1,2,011_S_0002,107,m36,m36,2008-08-27,NaN,1.0,NaN
3907,ADNIGO,2,011_S_0002,8,m60,m60,2010-09-27,1.0,NaN,NaN
4701,ADNI2,2,011_S_0002,8,v06,m72,2011-09-19,1.0,NaN,NaN
6465,ADNI2,2,011_S_0002,8,v11,m84,2012-10-04,4.0,NaN,NaN
7949,ADNI2,2,011_S_0002,8,v21,m96,2013-09-23,1.0,NaN,NaN
9453,ADNI2,2,011_S_0002,8,v41,m120,2015-10-20,1.0,NaN,NaN
9885,ADNI2,2,011_S_0002,8,v51,m132,2016-10-03,4.0,NaN,NaN
10303,ADNI3,2,011_S_0002,8,init,m144,2017-11-01,NaN,NaN,2.0


We notice that the majority of the rows have some diagnostic information in at least one of the columns: DXCHANGE, DXCURREN or DIAGNOSIS. We conclude that DXCURRENT is the diagnosis in ADNI1, DXCHANGE marks transitions though ADNI2 and DIAGNOSIS is diagnosis for ADNI3. It would be easier if each row had its own straightforward diagnosis, so we will infer it.    

If a row's diagnosis remains NaN, we will remove it, since we cannot use it for categorization.   

We will make an inferred column DX, with current diagnosis for each row (we cannot use 'DIAGNOSIS', since it only shows diagnosis specific for ADNI3).

In [58]:
def no_diagnosis(df):
    current = df['DXCURREN']
    change = df['DXCHANGE']
    diagnosis = df['DIAGNOSIS']
    
    # if diagnosis is not nan, dx = diagnosis
    if not math.isnan(diagnosis):
        return diagnosis
    # if current is not nan, dx = current
    if not math.isnan(current):
        return current
    # if change is nan, retain nan to remove rows later, otherwise ...
    if math.isnan(change):
        return change
    
    # map diagnostic values from DXCHANGE to 1=NC, 2=MCI, 3=AD
    mapping = {
        1: 1,
        2: 2,
        3: 3,
        4: 2,
        5: 3,
        6: 3,
        7: 1,
        8: 1,
        9: 1
    }
    
    return mapping[change]
           
# diag.apply(no_diagnosis, axis=1)

In [59]:
# Make a new column, which will have diagnosis for each row.
diag['DX'] = diag.apply(no_diagnosis, axis=1)
# diag.head()

In [60]:
# Remove rows where we could not infer diagnosis.
diag = diag.dropna(subset=['DX'])
diag['DX'].isnull().sum()

0

In [61]:
diag.reset_index(drop=True)

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DXCURREN,DIAGNOSIS,DX
0,ADNI1,2,011_S_0002,107,bl,bl,2005-09-29,NaN,1.0,NaN,1.0
1,ADNI1,2,011_S_0002,107,m06,m06,2006-03-06,NaN,1.0,NaN,1.0
2,ADNI1,3,011_S_0003,107,bl,bl,2005-09-30,NaN,3.0,NaN,3.0
3,ADNI1,3,011_S_0003,107,m06,m06,2006-03-13,NaN,3.0,NaN,3.0
4,ADNI1,3,011_S_0003,107,m12,m12,2006-09-13,NaN,3.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...
12980,ADNI3,7045,021_S_7045,16,sc,sc,2022-02-17,NaN,NaN,1.0,1.0
12981,ADNI3,7049,035_S_7049,25,sc,sc,2022-03-01,NaN,NaN,2.0,2.0
12982,ADNI3,6870,031_S_6870,22,sc,sc,2022-03-03,NaN,NaN,1.0,1.0
12983,ADNI3,7050,035_S_7050,25,sc,sc,2022-03-03,NaN,NaN,1.0,1.0


Now we have enough demographic information (age, gender, education, marital status, race, ethnicity, etc., as well as diagnosis for each row. We can join any desired column to our test dataframes.

In [62]:
diag.head()

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DXCURREN,DIAGNOSIS,DX
0,ADNI1,2,011_S_0002,107,bl,bl,2005-09-29,NaN,1.0,NaN,1.0
1,ADNI1,2,011_S_0002,107,m06,m06,2006-03-06,NaN,1.0,NaN,1.0
2,ADNI1,3,011_S_0003,107,bl,bl,2005-09-30,NaN,3.0,NaN,3.0
3,ADNI1,3,011_S_0003,107,m06,m06,2006-03-13,NaN,3.0,NaN,3.0
4,ADNI1,3,011_S_0003,107,m12,m12,2006-09-13,NaN,3.0,NaN,3.0


In [63]:
change = diag[diag['DXCHANGE'].isna()]
change['Phase'].unique()

array(['ADNI1', 'ADNI3'], dtype=object)

DXCHANGE has NaN values, where observation of diagnostic change is not yet possible: in the first phase of ADNI and with new participants in ADNI3. We will introduce new category - where no change could be observed yet, we will encode it with 0. This will make this column easier to work with and filter.    
The information from DXCURREN and DIAGNOSIS is now stored in DX and it is linked to Phase, RID and EXAMDATE, so it is uniquely defined for each entry and we can drop separate columns DXCURRENT and DIAGNOSIS - they stored the same type of info anyway, the difference being only, that DXCURREN had data on ADNI1 and DIAGNOSIS on ADNI3.

In [64]:
# Where change of diagnosis is not applicable, because the time period is too short, participants are new, etc.
diag['DXCHANGE'] =  diag['DXCHANGE'].fillna('0')

In [65]:
# Remove columns.
diag = diag.drop(['DXCURREN', 'DIAGNOSIS'], axis = 1)

In [66]:
diag.isnull().sum()

Phase        0
RID          0
PTID         0
SITEID       0
VISCODE      0
VISCODE2     3
EXAMDATE    19
DXCHANGE     0
DX           0
dtype: int64

In [67]:
diag[diag['VISCODE2'].isna()]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
11618,ADNI3,4037,041_S_4037,28,y2,NaN,2019-05-09,0,1.0
12792,ADNI3,5282,082_S_5282,37,y2,NaN,2020-10-20,0,2.0
12995,ADNI3,6266,033_S_6266,24,y4,NaN,2022-03-04,0,1.0


In [68]:
# Check what neighbouring participants have for VISCODE.
# For the participant RID = 6195
diag.iloc[11612:11620]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
11615,ADNI3,4419,021_S_4419,16,y2,m90,2019-05-13,0,1.0
11616,ADNI3,5097,041_S_5097,28,y1,m72,2019-04-25,0,1.0
11617,ADNI3,6690,082_S_6690,37,bl,bl,2019-05-16,0,3.0
11618,ADNI3,4037,041_S_4037,28,y2,NaN,2019-05-09,0,1.0
11619,ADNI3,4199,116_S_4199,44,y1,m90,2019-04-22,0,2.0
11620,ADNI3,6704,129_S_6704,49,bl,bl,2019-05-22,0,1.0
11621,ADNI3,1378,141_S_1378,55,y2,m144,2019-06-03,0,2.0
11622,ADNI3,6008,141_S_6008,55,y2,m24,2019-06-03,0,1.0


In [69]:
# Check what visits has this patient already had. What comes after VISCODE2 'm72'?
diag[diag['RID'] == 4037]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
4381,ADNI2,4037,041_S_4037,27,v01,sc,2011-04-29,1.0,1.0
4473,ADNI2,4037,041_S_4037,27,v03,bl,2011-06-27,1.0,1.0
5176,ADNI2,4037,041_S_4037,27,v05,m06,2012-01-24,1.0,1.0
6151,ADNI2,4037,041_S_4037,27,v11,m12,2012-07-25,1.0,1.0
7650,ADNI2,4037,041_S_4037,27,v21,m24,2013-07-05,1.0,1.0
9404,ADNI2,4037,041_S_4037,27,v41,m48,2015-09-16,1.0,1.0
10133,ADNI3,4037,041_S_4037,28,init,m72,2017-07-13,0,1.0
11618,ADNI3,4037,041_S_4037,28,y2,NaN,2019-05-09,0,1.0


In [70]:
# VISCODE2 == difference from penultimate visit in 'EXAMDATE', 2017-07-13, with visit code 'm72'
# until the visit with the missing VISCODE2 on 2019-05-09, which is ussually 'm96'
diag[diag['VISCODE2'] == 'm96'].head()

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
7948,ADNI2,8,011_S_0008,8,v21,m96,2013-09-23,1.0,1.0
7949,ADNI2,2,011_S_0002,8,v21,m96,2013-09-23,1.0,1.0
8109,ADNI2,31,023_S_0031,17,v21,m96,2013-10-21,1.0,1.0
8124,ADNI2,21,011_S_0021,8,v21,m96,2013-11-06,1.0,1.0
8192,ADNI2,42,023_S_0042,17,v31,m96,2013-12-04,3.0,3.0


In [71]:
# Check what visits other two patients already had?
print(diag[diag['RID'] == 5282]['VISCODE2'])
print(diag[diag['RID'] == 6266]['VISCODE2'])

7813      sc
8381      bl
8655     m06
9978     m24
11062    m54
11725    m72
12792    NaN
Name: VISCODE2, dtype: object
10563     sc
10765     bl
12221    m24
12995    NaN
Name: VISCODE2, dtype: object


In [72]:
# VISCODE2 for RID 4037
diag.at[11618, 'VISCODE2'] = 'm96'
# for RID 5282
diag.at[12792, 'VISCODE2'] = 'm96'
# for RID 6266
diag.at[12995, 'VISCODE2'] = 'm48'

diag[diag['VISCODE2'].isna()]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX


In [73]:
# # Is the last visit with VISCODE2 'm48'?
diag[diag['RID'] == 6266]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
10563,ADNI3,6266,033_S_6266,24,sc,sc,2018-03-07,0,1.0
10765,ADNI3,6266,033_S_6266,24,bl,bl,2018-05-18,0,1.0
12221,ADNI3,6266,033_S_6266,24,y2,m24,2020-03-17,0,1.0
12995,ADNI3,6266,033_S_6266,24,y4,m48,2022-03-04,0,1.0


In [74]:
# Assign order to Phases.
phase_order = ['ADNI1', 'ADNIGO', 'ADNI2', 'ADNI3']
diag['Phase'] = diag['Phase'].astype(CategoricalDtype(categories=phase_order, ordered=True))

In [75]:
diag = diag.sort_values(by = ['Phase', 'VISCODE', 'VISCODE2', 'RID']).reset_index(drop=True)
diag.tail(10)

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
12975,ADNI3,5132,127_S_5132,47,y4,m96,2021-05-27,0,2.0
12976,ADNI3,5169,027_S_5169,20,y4,m96,2021-05-18,0,1.0
12977,ADNI3,5170,027_S_5170,20,y4,m96,2021-06-17,0,1.0
12978,ADNI3,5177,021_S_5177,16,y4,m96,2021-06-16,0,1.0
12979,ADNI3,5200,127_S_5200,47,y4,m96,2021-10-05,0,1.0
12980,ADNI3,5222,037_S_5222,27,y4,m96,2021-06-23,0,1.0
12981,ADNI3,5237,021_S_5237,16,y4,m96,2021-10-01,0,3.0
12982,ADNI3,5265,007_S_5265,5,y4,m96,2021-09-21,0,2.0
12983,ADNI3,4919,027_S_4919,20,y5,m114,2022-01-31,0,2.0
12984,ADNI3,2245,027_S_2245,20,y5,m132,2022-01-27,0,2.0


In [76]:
NaT = diag[diag['EXAMDATE'].isna()].sort_values(by = ['Phase', 'VISCODE', 'RID'])
NaT

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
10123,ADNI3,6195,035_S_6195,25,bl,bl,NaT,0,1.0
10335,ADNI3,6512,127_S_6512,47,bl,bl,NaT,0,2.0
10712,ADNI3,752,116_S_0752,44,init,m144,NaT,0,3.0
11028,ADNI3,2396,053_S_2396,31,init,m90,NaT,0,2.0
10776,ADNI3,5285,051_S_5285,29,init,m54,NaT,0,2.0
10777,ADNI3,5294,051_S_5294,29,init,m54,NaT,0,2.0
11051,ADNI3,6024,127_S_6024,47,sc,sc,NaT,0,1.0
11122,ADNI3,6102,019_S_6102,14,sc,sc,NaT,0,3.0
11538,ADNI3,6599,137_S_6599,54,sc,sc,NaT,0,1.0
11561,ADNI3,6628,137_S_6628,54,sc,sc,NaT,0,1.0


In [77]:
NaT['VISCODE'].unique()

array(['bl', 'init', 'sc', 'y1', 'y2', 'y3'], dtype=object)

All missing dates occur in ADNI3.

Can we add approximate dates of exam based on Phase, VISCODE, and neighbouring entries? Since we have VISCODE and VISCODE2for all patients, we can calculate the approximate time of the visit +/- month. or our purposes, +/- 2 months doesnt play a role, we just want to preserve integrity of an individual's progression and are not interested in the exact dates.

In [78]:
diag[diag['RID'] == 6195].sort_values(by = ['Phase', 'EXAMDATE', 'VISCODE'])

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
11204,ADNI3,6195,035_S_6195,25,sc,sc,2018-01-24,0,1.0
10123,ADNI3,6195,035_S_6195,25,bl,bl,NaT,0,1.0


Thie invsetigation below gives us an idea whithin which timeframe the exam should happen: we checked individual prograssion aboven now we check the neighbourting entries below.

In [79]:
# Nearby entries for RIDs 6195
diag[diag['RID'] == 6192].sort_values(by = ['Phase', 'EXAMDATE'])

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
11203,ADNI3,6192,041_S_6192,28,sc,sc,2018-01-29,0,1.0
10122,ADNI3,6192,041_S_6192,28,bl,bl,2018-02-20,0,1.0
12475,ADNI3,6192,041_S_6192,28,y2,m24,2020-03-23,0,1.0


In [80]:
# Let's check dates for 2 participants close to 6195 who have all dates.
diag[diag['RID'] == 6197].sort_values(by = ['Phase', 'EXAMDATE'])

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
11205,ADNI3,6197,082_S_6197,37,sc,sc,2018-02-14,0,1.0
10124,ADNI3,6197,082_S_6197,37,bl,bl,2018-07-10,0,1.0


We notice two things. The person 6195 had screening in Jan 2018 for ADNI3. The neighbouring RIDs 6192 and 6197 had baseline visits between 2018-02-20 and 2018-07-10.

With na informed guess we can say that the participant with RID = 6195 was probably examined sometime in 2018 between February and October. (Phase + VISCODE/VISCODE2 are pretty strong indicators), since there is only one baseline exam per individual per Phase.)

In [81]:
diag = diag.sort_values(by = ['Phase', 'EXAMDATE', 'VISCODE', 'RID']).reset_index(drop=True)
diag.tail(25)

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
12960,ADNI3,7049,035_S_7049,25,sc,sc,2022-03-01,0,2.0
12961,ADNI3,1222,007_S_1222,5,y4,m180,2022-03-01,0,1.0
12962,ADNI3,6870,031_S_6870,22,sc,sc,2022-03-03,0,1.0
12963,ADNI3,7050,035_S_7050,25,sc,sc,2022-03-03,0,1.0
12964,ADNI3,6266,033_S_6266,24,y4,m48,2022-03-04,0,1.0
12965,ADNI3,1418,041_S_1418,28,y2,m144,2109-07-26,0,2.0
12966,ADNI3,6195,035_S_6195,25,bl,bl,NaT,0,1.0
12967,ADNI3,6512,127_S_6512,47,bl,bl,NaT,0,2.0
12968,ADNI3,752,116_S_0752,44,init,m144,NaT,0,3.0
12969,ADNI3,2396,053_S_2396,31,init,m90,NaT,0,2.0


For the participant at iloc[12965] we notice a strange (typo) date: 2109-07-26. Probably the yar should be 2019, but lets check the progression:

In [82]:
diag[diag['RID'] == 1418]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
1813,ADNI1,1418,041_S_1418,20,bl,bl,2007-08-14,0,2.0
2386,ADNI1,1418,041_S_1418,20,m06,m06,2008-02-18,0,2.0
2819,ADNI1,1418,041_S_1418,20,m12,m12,2008-07-28,0,2.0
3222,ADNI1,1418,041_S_1418,20,m18,m18,2009-02-24,0,2.0
3480,ADNI1,1418,041_S_1418,20,m24,m24,2009-08-11,0,2.0
3842,ADNI1,1418,041_S_1418,20,m36,m36,2010-09-17,0,2.0
5105,ADNI2,1418,041_S_1418,27,v06,m48,2011-12-22,2.0,2.0
6249,ADNI2,1418,041_S_1418,27,v11,m60,2012-08-09,2.0,2.0
8352,ADNI2,1418,041_S_1418,27,v21,m72,2013-12-23,2.0,2.0
8979,ADNI2,1418,041_S_1418,27,v31,m84,2014-10-17,2.0,2.0


The date should definitely be 2019-07-26, let's fix that.

In [83]:
diag.at[12965, 'EXAMDATE'] = '2019-07-26'

In [84]:
# Fix date type (We messed it up with str type insert.)
diag['EXAMDATE'] = pd.to_datetime(diag['EXAMDATE']).dt.normalize()

In [85]:
# Continue fixing missing date. Compare by phase, site, viscode and viscode2 and fill examdate with similar.
diag = diag.sort_values(by = ['Phase', 'VISCODE', 'VISCODE2', 'RID']).reset_index(drop=True)
diag[diag['EXAMDATE'].isna()] 

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
10123,ADNI3,6195,035_S_6195,25,bl,bl,NaT,0,1.0
10335,ADNI3,6512,127_S_6512,47,bl,bl,NaT,0,2.0
10712,ADNI3,752,116_S_0752,44,init,m144,NaT,0,3.0
10776,ADNI3,5285,051_S_5285,29,init,m54,NaT,0,2.0
10777,ADNI3,5294,051_S_5294,29,init,m54,NaT,0,2.0
11028,ADNI3,2396,053_S_2396,31,init,m90,NaT,0,2.0
11051,ADNI3,6024,127_S_6024,47,sc,sc,NaT,0,1.0
11122,ADNI3,6102,019_S_6102,14,sc,sc,NaT,0,3.0
11538,ADNI3,6599,137_S_6599,54,sc,sc,NaT,0,1.0
11561,ADNI3,6628,137_S_6628,54,sc,sc,NaT,0,1.0




We will fill the rows with missing date with the visit date from the row below.

In [86]:
# For the participant RID = 6195
diag.iloc[10120:10130]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
10120,ADNI3,6188,305_S_6188,62,bl,bl,2018-03-29,0,1.0
10121,ADNI3,6189,036_S_6189,26,bl,bl,2018-02-28,0,1.0
10122,ADNI3,6192,041_S_6192,28,bl,bl,2018-02-20,0,1.0
10123,ADNI3,6195,035_S_6195,25,bl,bl,NaT,0,1.0
10124,ADNI3,6197,082_S_6197,37,bl,bl,2018-07-10,0,1.0
10125,ADNI3,6199,014_S_6199,11,bl,bl,2018-03-28,0,1.0
10126,ADNI3,6200,035_S_6200,25,bl,bl,2018-04-02,0,1.0
10127,ADNI3,6202,024_S_6202,19,bl,bl,2018-03-07,0,1.0
10128,ADNI3,6203,127_S_6203,47,bl,bl,2018-04-25,0,1.0
10129,ADNI3,6204,037_S_6204,27,bl,bl,2018-07-10,0,1.0


In [87]:
# For the participant RID = 4485
diag.iloc[12225:12232]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
12225,ADNI3,4400,018_S_4400,13,y1,m78,2018-10-05,0,2.0
12226,ADNI3,4405,130_S_4405,50,y1,m78,2018-10-11,0,2.0
12227,ADNI3,4417,130_S_4417,50,y1,m78,2019-05-10,0,2.0
12228,ADNI3,4446,135_S_4446,52,y1,m78,2018-09-11,0,2.0
12229,ADNI3,4485,006_S_4485,4,y1,m78,NaT,0,1.0
12230,ADNI3,4488,007_S_4488,5,y1,m78,2018-10-25,0,1.0
12231,ADNI3,4489,135_S_4489,52,y1,m78,2018-06-01,0,2.0


In [88]:
diag[diag['RID'] == 4485]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
4747,ADNI2,4485,006_S_4485,4,v01,sc,2012-01-23,1.0,1.0
5713,ADNI2,4485,006_S_4485,4,v03,bl,2012-03-01,1.0,1.0
6477,ADNI2,4485,006_S_4485,4,v05,m06,2012-10-03,1.0,1.0
7571,ADNI2,4485,006_S_4485,4,v11,m12,2013-04-23,1.0,1.0
8488,ADNI2,4485,006_S_4485,4,v21,m24,2014-03-27,1.0,1.0
9772,ADNI2,4485,006_S_4485,4,v41,m48,2016-03-22,1.0,1.0
10852,ADNI3,4485,006_S_4485,4,init,m66,2017-09-21,0,1.0
12229,ADNI3,4485,006_S_4485,4,y1,m78,NaT,0,1.0
12723,ADNI3,4485,006_S_4485,4,y2,m90,2020-01-15,0,1.0


In [89]:
# Lets fill with the next date from the same phase and viscode.
diag['EXAMDATE'] = diag['EXAMDATE'].bfill()

In [90]:
# Check, if it makes sense for an RID with previously missing dates.
diag[diag['RID'] == 4485]

,Phase,RID,PTID,SITEID,VISCODE,VISCODE2,EXAMDATE,DXCHANGE,DX
4747,ADNI2,4485,006_S_4485,4,v01,sc,2012-01-23,1.0,1.0
5713,ADNI2,4485,006_S_4485,4,v03,bl,2012-03-01,1.0,1.0
6477,ADNI2,4485,006_S_4485,4,v05,m06,2012-10-03,1.0,1.0
7571,ADNI2,4485,006_S_4485,4,v11,m12,2013-04-23,1.0,1.0
8488,ADNI2,4485,006_S_4485,4,v21,m24,2014-03-27,1.0,1.0
9772,ADNI2,4485,006_S_4485,4,v41,m48,2016-03-22,1.0,1.0
10852,ADNI3,4485,006_S_4485,4,init,m66,2017-09-21,0,1.0
12229,ADNI3,4485,006_S_4485,4,y1,m78,2018-10-25,0,1.0
12723,ADNI3,4485,006_S_4485,4,y2,m90,2020-01-15,0,1.0


I nthe above cell we checked, if new date makes sense for the RID with previously missing exam date. VISCODE2 is m78, which is 12 months after VISCODE m66. We see that the visits are approx. 12 months apart (also for the examdates where original date is available, there is a 1 months +/- deviation).

In [91]:
diag.isnull().sum()

Phase       0
RID         0
PTID        0
SITEID      0
VISCODE     0
VISCODE2    0
EXAMDATE    0
DXCHANGE    0
DX          0
dtype: int64

In [92]:
diag.shape

(12985, 9)

In [93]:
# How many entries did we lose to cleaning?
diag_finish_entries = diag.shape[0]
dropped_entries = diag_start_entries - diag_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/diag_start_entries*100, 2), '% removed')

11
0.08 % removed


In [94]:
# Writing diag to csv.
df = diag
df.to_csv('../../data/tabular_data/clean_DXSUM_ADNIALL.csv', index=False)

### <a id="tests">2. Neuropsychological tests - cleaning</a>  

### Clinical Dementia Rating (CDR)

In [95]:
# CDR data dictionary, for information about the abbreviations in column names.
cdr_dict = pd.read_csv('../../data/tabular_data/CDR_dict.csv', header=None)
cdr_dict.style.set_properties(**{'text-align': 'left'}).hide_index()

0,1
Phase,Phase
ID,Record ID
RID,Participant roster ID
SITEID,Site ID
VISCODE,Visit code
VISCODE2,Translated visit code: : longitudinal progression
USERDATE,Date record created
USERDATE2,Date record last updated
EXAMDATE,Examination Date
CDSOURCE,Information Source: 1=Participant Visit;2=Telephone Call


Each question is evaluated with 0, 0.5, 1, 2, or 3 points.

In [96]:
# Clinical dementia rating scale
df = pd.read_csv('../../data/tabular_data/CDR.csv')
print(df.info())
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12305 entries, 0 to 12304
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Phase         12305 non-null  object 
 1   ID            12305 non-null  int64  
 2   RID           12305 non-null  int64  
 3   SITEID        12305 non-null  int64  
 4   VISCODE       12305 non-null  object 
 5   VISCODE2      12278 non-null  object 
 6   USERDATE      12305 non-null  object 
 7   USERDATE2     6939 non-null   object 
 8   EXAMDATE      4205 non-null   object 
 9   CDSOURCE      12196 non-null  float64
 10  CDVERSION     7387 non-null   float64
 11  CDMEMORY      12272 non-null  float64
 12  CDORIENT      12272 non-null  float64
 13  CDJUDGE       12272 non-null  float64
 14  CDCOMMUN      12272 non-null  float64
 15  CDHOME        12272 non-null  float64
 16  CDCARE        12271 non-null  float64
 17  CDGLOBAL      12271 non-null  float64
 18  CDRSB         2395 non-nul

(12305, 21)

In [97]:
# Columns that can be directly dropped, since the information is irrelevant.
drop_cols = ['SITEID', 'USERDATE', 'USERDATE2', 'CDSOURCE', 'CDVERSION', 'update_stamp']

df = df.drop(columns=drop_cols, axis=1)
cdr = df

In [98]:
cdr_start_entries = cdr.shape[0]

In [99]:
cdr.isnull().sum()

Phase          0
ID             0
RID            0
VISCODE        0
VISCODE2      27
EXAMDATE    8100
CDMEMORY      33
CDORIENT      33
CDJUDGE       33
CDCOMMUN      33
CDHOME        33
CDCARE        34
CDGLOBAL      34
CDRSB       9910
CDSOB       9910
dtype: int64

In [100]:
cdr.head()

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDRSB,CDSOB
0,ADNI1,8,2,sc,sc,2005-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,ADNI1,10,3,sc,sc,2005-08-18,1.0,1.0,1.0,1.0,0.5,0.0,1.0,NaN,NaN
2,ADNI1,12,4,sc,sc,2005-08-18,0.5,0.0,0.5,0.0,0.0,0.0,0.5,NaN,NaN
3,ADNI1,14,5,sc,sc,2005-08-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,ADNI1,16,7,sc,sc,2005-08-25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN


In [101]:
# Remove rows with missing test input.
cdr_score_columns = ['CDMEMORY', 'CDORIENT', 'CDJUDGE', 'CDCOMMUN', 'CDHOME', 'CDCARE']

# Drop rows with NA values. Drop only those with missing scores, missing visit dates etc. play no role.
initial_count = len(cdr)
cdr = cdr.dropna(subset=cdr_score_columns).reset_index(drop=True)
dropped_count = initial_count - len(cdr)
print('Total dropped records: ', dropped_count)
# cdr.isnull().sum()

Total dropped records:  34


'CDRSB' is a special score. According to the [Knight Alzheimer Disease Research Center](https://knightadrc.wustl.edu/professionals-clinicians/cdr-dementia-staging-instrument/sum-of-boxes/), the "Sum of Boxes provides a more quantitative measure of dementia severity" than the global CDR. But there are particular rules for determining CDRSB, that we don't know. The missing values in the column 'sum of boxes' ensue as a consequence.   

Global CDR involves a scoring algorithm that weights memory as the primary domain and all other domains as secondary. A rating of 0 indicates no cognitive impairment and a rating of 1, 2, or 3 indicates mild, moderate, or severe dementia. We were not able to obtain the algorithm in a timely manner to calculate this filed on ouw own, and since "this approach has been critiqued as being a relatively coarse estimation of dementia severity' [source](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3557804/), we decided to disregard it. 

CDSOB - Clinical Dementia Sum of Boxes is sum of all entered values in a row. We cannot calculate it, because we are missing values from CDRGLOBAL. To obtain some kind of total score anyway, we'll make a new colmn and simply sum up the values in the previous columns.

In [102]:
# Sum all the scores. 
cdr['total_score'] = 0
for col in cdr_score_columns:
    cdr['total_score'] += cdr[col]

cdr.head()

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDRSB,CDSOB,total_score
0,ADNI1,8,2,sc,sc,2005-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1,ADNI1,10,3,sc,sc,2005-08-18,1.0,1.0,1.0,1.0,0.5,0.0,1.0,NaN,NaN,4.5
2,ADNI1,12,4,sc,sc,2005-08-18,0.5,0.0,0.5,0.0,0.0,0.0,0.5,NaN,NaN,1.0
3,ADNI1,14,5,sc,sc,2005-08-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
4,ADNI1,16,7,sc,sc,2005-08-25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,6.0


In [103]:
# Fix missing exam date by merging 'EXAMDATE' from REGISTRY.csv on 'ID' in df
# Preserve sequence of exams by grouping by Phase and VISCODE
reg = pd.read_csv('../../data/tabular_data/REGISTRY.csv')
reg = reg[(~reg['EXAMDATE'].isnull() == True)] 


# Missing values in EXAMDATE?
print(reg['EXAMDATE'].isnull().sum())
reg = reg.sort_values(by='RID').reset_index(drop=True)
examdate = reg[['Phase', 'RID', 'VISCODE', 'EXAMDATE']]
examdate.head(20)

0


,Phase,RID,VISCODE,EXAMDATE
0,ADNI1,1,f,2005-08-18
1,ADNI1,2,sc,2005-08-17
2,ADNI2,2,v51,2016-09-27
3,ADNI2,2,v42,2016-06-23
4,ADNI2,2,v06,2011-09-19
5,ADNI2,2,v41,2015-09-22
6,ADNI2,2,v31,2014-10-13
7,ADNIGO,2,m66,2011-03-04
8,ADNI2,2,v11,2012-09-26
9,ADNI2,2,v22,2014-03-17


In [104]:
# Remove participants who failed screening (VISCODE = 'f')
cdr = cdr[cdr.VISCODE != 'f']
cdr

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDRSB,CDSOB,total_score
0,ADNI1,8,2,sc,sc,2005-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1,ADNI1,10,3,sc,sc,2005-08-18,1.0,1.0,1.0,1.0,0.5,0.0,1.0,NaN,NaN,4.5
2,ADNI1,12,4,sc,sc,2005-08-18,0.5,0.0,0.5,0.0,0.0,0.0,0.5,NaN,NaN,1.0
3,ADNI1,14,5,sc,sc,2005-08-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
4,ADNI1,16,7,sc,sc,2005-08-25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12266,ADNI3,138404,6726,y1,m24,NaN,0.5,0.0,0.5,0.5,0.5,1.0,0.5,3.0,3.0,3.0
12267,ADNI3,138519,2333,y3,m120,NaN,0.5,0.0,0.5,0.0,0.0,0.0,0.5,1.0,1.0,1.0
12268,ADNI3,138481,4598,y4,m108,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12269,ADNI3,138460,6949,sc,NaN,NaN,0.5,0.5,0.5,0.0,0.0,0.0,0.5,1.5,1.5,1.5


In [105]:
# Fix missing date.
cdr = pd.merge(cdr, examdate, how='left', on=['Phase', 'RID', 'VISCODE'])
cdr['EXAMDATE_y'].isnull().sum()

15

In [106]:
cdr[cdr['EXAMDATE_y'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDRSB,CDSOB,total_score,EXAMDATE_y
2704,ADNI1,6088,243,m24,m24,2008-06-04,2.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,8.0,NaN
3615,ADNI1,7912,1430,m36,m36,2009-10-01,2.0,3.0,2.0,2.0,2.0,1.0,2.0,NaN,NaN,12.0,NaN
3851,ADNI1,8388,1407,m12,m12,2008-09-03,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,-6.0,NaN
3930,ADNIGO,1232,830,m60,m60,NaN,1.0,0.5,1.0,1.0,0.5,0.0,0.5,NaN,NaN,4.0,NaN
8932,ADNI2,8966,5256,v11,m12,NaN,0.0,0.0,0.5,0.0,0.0,0.0,0.0,NaN,NaN,0.5,NaN
10344,ADNI3,51095,2119,y1,NaN,NaN,1.0,1.0,0.5,0.0,0.0,0.0,0.5,2.5,2.5,2.5,NaN
10578,ADNI3,67266,4414,y1,NaN,NaN,3.0,3.0,3.0,2.0,2.0,0.0,2.0,13.0,14.0,13.0,NaN
10862,ADNI3,101671,981,y2,m156,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
11604,ADNI3,121701,6375,y2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
11615,ADNI3,122097,6252,y2,NaN,NaN,1.0,0.5,1.0,1.0,0.5,0.0,0.5,4.0,4.0,4.0,NaN


In [107]:
cdr.at[3033, 'EXAMDATE_y'] = cdr['EXAMDATE_x']
cdr.at[3944, 'EXAMDATE_y'] = cdr['EXAMDATE_x']
cdr.at[4180, 'EXAMDATE_y'] = cdr['EXAMDATE_x']
cdr[cdr['EXAMDATE_y'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDRSB,CDSOB,total_score,EXAMDATE_y
2704,ADNI1,6088,243,m24,m24,2008-06-04,2.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,8.0,NaN
3615,ADNI1,7912,1430,m36,m36,2009-10-01,2.0,3.0,2.0,2.0,2.0,1.0,2.0,NaN,NaN,12.0,NaN
3851,ADNI1,8388,1407,m12,m12,2008-09-03,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,-6.0,NaN
3930,ADNIGO,1232,830,m60,m60,NaN,1.0,0.5,1.0,1.0,0.5,0.0,0.5,NaN,NaN,4.0,NaN
8932,ADNI2,8966,5256,v11,m12,NaN,0.0,0.0,0.5,0.0,0.0,0.0,0.0,NaN,NaN,0.5,NaN
10344,ADNI3,51095,2119,y1,NaN,NaN,1.0,1.0,0.5,0.0,0.0,0.0,0.5,2.5,2.5,2.5,NaN
10578,ADNI3,67266,4414,y1,NaN,NaN,3.0,3.0,3.0,2.0,2.0,0.0,2.0,13.0,14.0,13.0,NaN
10862,ADNI3,101671,981,y2,m156,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
11604,ADNI3,121701,6375,y2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
11615,ADNI3,122097,6252,y2,NaN,NaN,1.0,0.5,1.0,1.0,0.5,0.0,0.5,4.0,4.0,4.0,NaN


In [108]:
cdr = cdr.drop('EXAMDATE_x', axis=1)
cdr.isnull().sum()

Phase             0
ID                0
RID               0
VISCODE           0
VISCODE2         24
CDMEMORY          0
CDORIENT          0
CDJUDGE           0
CDCOMMUN          0
CDHOME            0
CDCARE            0
CDGLOBAL          1
CDRSB          9547
CDSOB          9548
total_score       0
EXAMDATE_y       15
dtype: int64

In [109]:
cdr.columns = cdr.columns.str.replace('EXAMDATE_y', 'EXAMDATE')
# cdr.tail(35)

In [110]:
cdr = cdr.dropna(subset=['EXAMDATE'])

For the ID=136425 we notice a possible wrong value in CDSOB, it is 10.0 instead of probably 1.0 (the only value in the entire row, so sum of boxes cannot be 10).   
For the rows, where the values in CDRSB and CDSOB are entered, they are sometimes different, sometimes equal. Search on the internet does not explain, how these columns are calculated, so we will drop them, as they have 9877/12269 missing values anyway.

In [111]:
cdr = cdr.drop(columns = ['CDRSB', 'CDSOB'])
cdr.isnull().sum()

Phase           0
ID              0
RID             0
VISCODE         0
VISCODE2       15
CDMEMORY        0
CDORIENT        0
CDJUDGE         0
CDCOMMUN        0
CDHOME          0
CDCARE          0
CDGLOBAL        1
total_score     0
EXAMDATE        0
dtype: int64

Each question can be scored with maximum 3 points. Let's check column values.

In [112]:
cdr = cdr.dropna(subset=['CDGLOBAL'])

In [113]:
viscode2 = reg[(reg['VISCODE2'].isnull() == True)] 
viscode2

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSTATUS,RGSTATUS,...,RGCONDCT,RGREASON,RGOTHSPE,RGSOURCE,RGRESCRN,RGPREVID,CHANGTR,CGTRACK,CGTRACK2,update_stamp
10812,ADNI2,23888,4028,26,tau,NaN,08/09/2017,NaN,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2017-08-09 19:51:41.0
12034,ADNI2,23196,4198,47,tau,NaN,09/30/2016,NaN,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2016-09-30 19:56:38.0
14635,ADNI2,23256,4552,35,tau,NaN,10/14/2016,NaN,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2016-10-14 19:56:48.0
17477,ADNI2,22668,5083,19,tau,NaN,06/29/2016,NaN,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2016-06-29 19:55:50.0
17679,ADNI2,23886,5126,26,tau,NaN,08/05/2017,NaN,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2017-08-05 19:51:38.0
17692,ADNI2,22666,5127,19,tau,NaN,06/29/2016,06/29/2016,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2016-06-29 19:55:50.0
18067,ADNI2,23366,5200,47,tau,NaN,11/15/2016,NaN,1.0,1.0,...,NaN,NaN,-4,3.0,NaN,-4,NaN,NaN,NaN,2016-11-15 19:57:44.0


In [114]:
# # Should be viscode in this case 'tau'? It is about the "tau-only visit", where they took the samples. 
cdr['VISCODE2'].unique().tolist()
# # Yes, seems valid.

['sc',
 'm06',
 'm12',
 'm36',
 'm18',
 'uns1',
 'm24',
 'm48',
 'm60',
 'm72',
 'm84',
 'm96',
 'm108',
 'm120',
 'm132',
 'm144',
 'm156',
 'm168',
 nan,
 'm180']

In [115]:
cdr['VISCODE2'] = cdr['VISCODE2'].fillna('tau')

In [116]:
# Other missing values - sometimes encoded as -1 or -4:
mis_mem = cdr[cdr.CDMEMORY == -1]
mis_mem

,Phase,ID,RID,VISCODE,VISCODE2,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,total_score,EXAMDATE
560,ADNI1,1556,48,m06,m06,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2006-06-15
709,ADNI1,8412,575,m36,m36,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2009-10-12
2096,ADNI1,4868,492,m06,m06,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2006-12-07
2206,ADNI1,5088,958,m06,m06,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2007-05-17
2262,ADNI1,5200,898,m12,m12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2007-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9537,ADNI2,10206,5202,v31,m36,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2016-06-22
9538,ADNI2,10208,5131,v21,m24,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2015-04-16
9541,ADNI2,10216,4050,v41,m48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2015-06-30
9542,ADNI2,10218,4343,v41,m48,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-6.0,2015-12-03


In [117]:
mis_mem = cdr[cdr.CDMEMORY == -4]
mis_mem

,Phase,ID,RID,VISCODE,VISCODE2,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,total_score,EXAMDATE


We forgot, that missing values are encoded differently in these datasets. For the folowing dataframes we will replace the hidden missing values right away with:     

df.replace(-1, np.NaN)    
df.replace(-4, np.NaN)

It appears that we have additional missing values encoded with -1 in more than 130 rows,where actually entire rows are useless. Since all the infromation on participant is missing, we will drop these rows.

In [118]:
before = cdr.shape[0]
before

11926

In [119]:
#CDGLOBAL is sum of other columns and will be negative.
cdr = cdr[cdr['CDGLOBAL'] >= 0]

In [120]:
# Did we remove all?
after = cdr.shape[0]
print(before - after)

132


In [121]:
cdr.shape

(11794, 14)

In [122]:
cdr.isnull().sum()

Phase          0
ID             0
RID            0
VISCODE        0
VISCODE2       0
CDMEMORY       0
CDORIENT       0
CDJUDGE        0
CDCOMMUN       0
CDHOME         0
CDCARE         0
CDGLOBAL       0
total_score    0
EXAMDATE       0
dtype: int64

In [123]:
cdr.shape

(11794, 14)

In [124]:
# How many entries did we lose to cleaning?
cdr_finish_entries = cdr.shape[0]
dropped_entries = cdr_start_entries - cdr_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/cdr_start_entries*100, 2), '% removed')

511
4.15 % removed


In [125]:
# Writing df to csv.
df = cdr
df.to_csv('../../data/tabular_data/clean_CDR.csv', index=False)

### Geriatric Depression Scale (GDS)

The Geriatric Depression Scale (GDS) is an assessment used to identify depression in the elderly. The scale consists of 15 yes/no questions. Answers in bold suggest depression, they are scored with 1 point:  

YES/**NO** 1. Are you basically satisfied with your life?  
**YES**/NO 2. Have you dropped many of your activities and interests?   
**YES**/NO 3. Do you feel that your life is empty?   
**YES**/NO 4. Do you often get bored?   
YES/**NO** 5. Are you in good spirits most of the time?    
**YES**/NO 6. Are you afraid that something bad is going to happen to you?   
YES/**NO** 7. Do you feel happy most of the time?   
**YES**/NO 8. Do you often feel helpless?   
**YES**/NO 9. Do you prefer to stay at home, rather than going out and doing new things?   
**YES**/NO 10. Do you feel you have more problems with memory than most?   
YES/**NO** 11. Do you think it is wonderful to be alive now?   
**YES**/NO 12. Do you feel pretty worthless the way you are now?   
YES/**NO** 13. Do you feel full of energy?    
**YES**/NO 14. Do you feel that your situation is hopeless?   
**YES**/NO 15. Do you think that most people are better off than you are?   

In the dataframe, the answer "yes" is noted as 1 and answer "no" is noted as 0.    
This might be a source of confusion: Only answers indicative of depression are scored as 1 point. Sometimes that does not correspond to the noted number in the field. Example: 0 (no) in 1. question is scored with 1 point (indicates depression), but 0 in 2. question is scored as 0 points (not dropping activities is a sign of health).

Higher total score (GDTOTAL) denotes more depressive symptoms. 
A score of 5 or more suggests depression.

**GDS, maximum score = 15:**

**0   -     4    normal  
5   -     8    mild  
9   -   11    moderate  
12 -   15    severe**

Alzheimer's disease participants: "findings suggest that depressive symptoms in AD patients are associated with cortical
thinning in temporal and parietal regions." (Lebedeva, 2014)

<u>Sources:<u>
- https://adni.loni.usc.edu/adni-publications/Lebedeva_2014_JNNP.pdf
- https://geriatrictoolkit.missouri.edu/cog/GDS_SHORT_FORM.PDF
- http://www.medafile.com/GDS15.htm

In [126]:
gds = pd.read_csv('../../data/tabular_data/GDSCALE.csv')
gds.shape

(11372, 29)

In [127]:
gds_dict = pd.read_csv('../../data/tabular_data/GDSCALE_dict.csv')
gds_dict.style.set_properties(**{'text-align': 'left'}).hide_index()
gds_dict

,Phase,Phase.1
0,ID,Record ID
1,RID,Participant roster ID
2,SITEID,Site ID
3,VISCODE,Visit code
4,VISCODE2,Translated visit code: longutudinal progression
5,USERDATE,Date record created
6,USERDATE2,Date record last updated
7,EXAMDATE,Examination Date
8,GDSOURCE,Information Source: 1=Participant Visit;2=Tele...
9,GDUNABL,1=Participant is unable to complete the GDS(Ge...


This is a questionnaire that scores 'yes' with 1 and 'no' with 0, but just to keep us on our toes, the value 1 in cell sometimes counts as 1 point and sometimes as 0 points. :)   

Only a subset of columns is interesting to us. If a participant was unable to complete the test, this might be relevant, so we will retain the column for now.

In [128]:
gds_cols = gds.columns.tolist()
print()
print(gds_cols)
print()
gds.info()


['Phase', 'ID', 'RID', 'SITEID', 'VISCODE', 'VISCODE2', 'USERDATE', 'USERDATE2', 'EXAMDATE', 'GDSOURCE', 'GDUNABL', 'GDUNABSP', 'GDSATIS', 'GDDROP', 'GDEMPTY', 'GDBORED', 'GDSPIRIT', 'GDAFRAID', 'GDHAPPY', 'GDHELP', 'GDHOME', 'GDMEMORY', 'GDALIVE', 'GDWORTH', 'GDENERGY', 'GDHOPE', 'GDBETTER', 'GDTOTAL', 'update_stamp']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11372 entries, 0 to 11371
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Phase         11372 non-null  object 
 1   ID            11372 non-null  int64  
 2   RID           11372 non-null  int64  
 3   SITEID        11372 non-null  int64  
 4   VISCODE       11372 non-null  object 
 5   VISCODE2      11341 non-null  object 
 6   USERDATE      11372 non-null  object 
 7   USERDATE2     2698 non-null   object 
 8   EXAMDATE      3130 non-null   object 
 9   GDSOURCE      11323 non-null  float64
 10  GDUNABL       11347 non-null  float64
 11  GDUN

In [129]:
# Columns we are interested in.
gds_columns = ['Phase', 'ID', 'RID', 'VISCODE', 'VISCODE2',  
               'GDUNABL', 'GDUNABSP', 'GDSATIS', 'GDDROP', 
               'GDEMPTY', 'GDBORED', 'GDSPIRIT', 'GDAFRAID',
               'GDHAPPY', 'GDHELP', 'GDHOME', 'GDMEMORY', 
               'GDALIVE', 'GDWORTH', 'GDENERGY', 'GDHOPE', 
               'GDBETTER', 'GDTOTAL', 'EXAMDATE']
gds = gds[gds_columns]

In [130]:
gds_start_entries = gds.shape[0]
gds_start_entries

11372

In [131]:
# How many participants were unable to complete the test?
print(gds['GDUNABL'].value_counts()[1])
x = np.round(gds['GDUNABL'].value_counts()[1] / gds.shape[0], 3)
print(x, '% of the participants were unable to complete the test.')

109
0.01 % of the participants were unable to complete the test.


In [132]:
gds[gds['GDUNABL'] == 1]

,Phase,ID,RID,VISCODE,VISCODE2,GDUNABL,GDUNABSP,GDSATIS,GDDROP,GDEMPTY,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
1419,ADNI1,2858,425,m12,m12,1.0,The nurse accidentally missed this form and di...,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2007-06-15
1435,ADNI1,2890,88,m12,m12,1.0,pt refused further testing - was becoming agit...,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2007-02-12
1591,ADNI1,3202,565,m12,m12,1.0,participant couldn't understand questions that...,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2007-08-02
1814,ADNI1,3650,93,m24,m24,1.0,unresponsive; lack of comprehension during int...,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2008-01-28
1965,ADNI1,3952,88,m24,m24,1.0,patient cannot respond approprately to questions,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2008-03-19
2397,ADNI1,4816,565,m24,m24,1.0,could not comprehend questions,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2008-08-07
2526,ADNI1,5074,1188,m24,m24,1.0,Administrative oversight,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2009-01-20
2613,ADNI1,5248,1382,m24,m24,1.0,"Questionnaire administered, sujbect couldn't a...",-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2009-04-22
2654,ADNI1,5332,1402,m24,m24,1.0,participant was too agitated and aggressive to...,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2009-05-28
2802,ADNI1,6268,1205,m24,m24,1.0,Not done/unknown,-4.0,-4.0,-4.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,2009-03-16


In [133]:
# Change hidden missing values.
gds = gds.replace(-1, np.NaN)    
gds = gds.replace(-4, np.NaN)

In [134]:
# For these we can't know - missing entry, or normal outcome, and thus nothing entered?
gds['GDUNABL'].isnull().sum()

8875

The number of participants, who could not complete the test is very small and the stated reasons are completely random 'The nurse accidentally missed this form'. We will just drop the columns.

In [135]:
gds = gds.drop(columns = ['GDUNABL', 'GDUNABSP'])

In [136]:
gds.isnull().sum()

Phase          0
ID             0
RID            0
VISCODE        0
VISCODE2      31
GDSATIS      160
GDDROP       160
GDEMPTY      161
GDBORED      160
GDSPIRIT     160
GDAFRAID     159
GDHAPPY      160
GDHELP       159
GDHOME       162
GDMEMORY     165
GDALIVE      162
GDWORTH      161
GDENERGY     160
GDHOPE       161
GDBETTER     162
GDTOTAL      171
EXAMDATE    8242
dtype: int64

In [137]:
gds = gds.dropna(subset=[
    'GDSATIS', 'GDDROP', 'GDEMPTY', 'GDBORED', 'GDSPIRIT',
    'GDAFRAID', 'GDHAPPY', 'GDHELP', 'GDHOME', 'GDMEMORY',
    'GDALIVE', 'GDWORTH', 'GDENERGY', 'GDHOPE', 'GDBETTER'
])

In [138]:
gds.isnull().sum()

Phase          0
ID             0
RID            0
VISCODE        0
VISCODE2      24
GDSATIS        0
GDDROP         0
GDEMPTY        0
GDBORED        0
GDSPIRIT       0
GDAFRAID       0
GDHAPPY        0
GDHELP         0
GDHOME         0
GDMEMORY       0
GDALIVE        0
GDWORTH        0
GDENERGY       0
GDHOPE         0
GDBETTER       0
GDTOTAL        4
EXAMDATE    8080
dtype: int64

In [139]:
total = gds[gds['GDTOTAL'].isna()]

In [140]:
gds['GDTOTAL'].unique()

array([ 1.,  3.,  0.,  2.,  5.,  4.,  8.,  7., 12., 11.,  6.,  9., 10.,
       13., 14., nan])

With some questions, "Yes" counts as 1 point, for other questions "No" counts as 1 point, according to the our data dictionary and the [Geriatric Depression Scale](https://www.vumc.org/cqa/sites/vumc.org.cqa/files/public_files/4%29%20Step%201%20-%20Geriatric%20Depression%20Scale.pdf). For these 4 entries we can map, which value in the df counts toward CDTOTAL and calcuate it.

In [141]:
cols = [
    'GDSATIS', 'GDDROP', 'GDEMPTY', 'GDBORED', 'GDSPIRIT',
    'GDAFRAID', 'GDHAPPY', 'GDHELP', 'GDHOME', 'GDMEMORY',
    'GDALIVE', 'GDWORTH', 'GDENERGY', 'GDHOPE', 'GDBETTER'
]

[total[cols]]

[       GDSATIS  GDDROP  GDEMPTY  GDBORED  GDSPIRIT  GDAFRAID  GDHAPPY  GDHELP  \
 10437      1.0     0.0      0.0      0.0       1.0       0.0      1.0     0.0   
 10708      1.0     1.0      0.0      1.0       1.0       0.0      1.0     0.0   
 10837      1.0     0.0      0.0      0.0       1.0       0.0      1.0     0.0   
 11036      1.0     1.0      0.0      0.0       1.0       0.0      1.0     0.0   
 
        GDHOME  GDMEMORY  GDALIVE  GDWORTH  GDENERGY  GDHOPE  GDBETTER  
 10437     0.0       0.0      1.0      0.0       1.0     0.0       0.0  
 10708     1.0       1.0      1.0      0.0       1.0     0.0       0.0  
 10837     1.0       0.0      1.0      0.0       1.0     0.0       0.0  
 11036     0.0       1.0      1.0      0.0       1.0     0.0       0.0  ]

In [142]:
# Which value in the table counts as +1 for the sum?
def cond_sum (x):
    mapping = {
        'GDSATIS':0,
        'GDDROP':1,
        'GDEMPTY':1,
        'GDBORED':1,
        'GDSPIRIT':0,
        'GDAFRAID':1,
        'GDHAPPY':0,
        'GDHELP':1,
        'GDHOME':1,
        'GDMEMORY':1,
        'GDALIVE':0,
        'GDWORTH':1,
        'GDENERGY':0,
        'GDHOPE':1,
        'GDBETTER':1        
    }
    
    result = 0
    for key, value in mapping.items():
        if x[key] == value:
            result += 1
    return result
        

gds[gds['GDTOTAL'].isna()].apply(cond_sum, axis=1)

10437    0
10708    4
10837    1
11036    2
dtype: int64

In [143]:
gds.at[10437, 'GDTOTAL'] = 0
gds.at[10708, 'GDTOTAL'] = 4
gds.at[10837, 'GDTOTAL'] = 1
gds.at[11036, 'GDTOTAL'] = 2

gds.isnull().sum()

Phase          0
ID             0
RID            0
VISCODE        0
VISCODE2      24
GDSATIS        0
GDDROP         0
GDEMPTY        0
GDBORED        0
GDSPIRIT       0
GDAFRAID       0
GDHAPPY        0
GDHELP         0
GDHOME         0
GDMEMORY       0
GDALIVE        0
GDWORTH        0
GDENERGY       0
GDHOPE         0
GDBETTER       0
GDTOTAL        0
EXAMDATE    8080
dtype: int64

In [144]:
# Merge exam dates.
gds = pd.merge(gds, examdate, how='left', on=['Phase', 'RID', 'VISCODE'])
gds.isnull().sum()

Phase            0
ID               0
RID              0
VISCODE          0
VISCODE2        24
GDSATIS          0
GDDROP           0
GDEMPTY          0
GDBORED          0
GDSPIRIT         0
GDAFRAID         0
GDHAPPY          0
GDHELP           0
GDHOME           0
GDMEMORY         0
GDALIVE          0
GDWORTH          0
GDENERGY         0
GDHOPE           0
GDBETTER         0
GDTOTAL          0
EXAMDATE_x    8080
EXAMDATE_y      14
dtype: int64

In [145]:
# Were the missing values again in ADNI3?
gds.tail()

,Phase,ID,RID,VISCODE,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE_x,EXAMDATE_y
11187,ADNI3,138306,6349,y2,m36,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,2021-07-20
11188,ADNI3,138290,6402,y3,m36,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,2021-07-20
11189,ADNI3,138518,2333,y3,m120,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,2021-07-26
11190,ADNI3,138480,4598,y4,m108,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,NaN,2021-07-20
11191,ADNI3,138614,6988,sc,sc,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,NaN,2021-07-27


In [146]:
# List of participants with irreparably missing date.
gds[gds['EXAMDATE_y'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE_x,EXAMDATE_y
2084,ADNI1,4200,243,m24,m24,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2008-06-04,NaN
2827,ADNI1,5700,1430,m36,m36,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,2009-10-01,NaN
3718,ADNIGO,1262,830,m60,m60,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN
8240,ADNI2,9106,5256,v11,m12,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN
8896,ADNI3,4527,907,init,m132,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
10418,ADNI3,100631,6214,sc,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
10435,ADNI3,101675,981,y2,m156,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,NaN,NaN
10587,ADNI3,109073,6220,y1,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
10739,ADNI3,116211,6375,y2,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN
10814,ADNI3,119133,6252,y2,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [147]:
gds.at[2084, 'EXAMDATE_y'] = gds['EXAMDATE_x']
gds.at[2827, 'EXAMDATE_y'] = gds['EXAMDATE_x']

In [148]:
gds = gds.drop('EXAMDATE_x', axis=1)
gds.columns = gds.columns.str.replace('EXAMDATE_y', 'EXAMDATE')

gds['EXAMDATE'].isnull().sum()

12

In [149]:
gds.isna().sum()

Phase        0
ID           0
RID          0
VISCODE      0
VISCODE2    24
GDSATIS      0
GDDROP       0
GDEMPTY      0
GDBORED      0
GDSPIRIT     0
GDAFRAID     0
GDHAPPY      0
GDHELP       0
GDHOME       0
GDMEMORY     0
GDALIVE      0
GDWORTH      0
GDENERGY     0
GDHOPE       0
GDBETTER     0
GDTOTAL      0
EXAMDATE    12
dtype: int64

We won't drop cells with missing exam dates, since there might be no need to use date in each analysis and we would loose 12 good entries. If date will be important, we can drop them later.

In [150]:
# Fill all the NaN VISCODE2 for multindex ['Phase', 'RID', 'VISCODE'] with the respective VISCODE2 in reg df.
gds = gds.set_index(['Phase', 'RID', 'VISCODE'])
gds['VISCODE2'] = gds['VISCODE2'].fillna(reg.set_index(['Phase', 'RID', 'VISCODE'])['VISCODE2'])

In [151]:
gds.reset_index(inplace=True)

In [152]:
gds[gds['VISCODE2'].isna()]

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
10418,ADNI3,6214,sc,100631,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10587,ADNI3,6220,y1,109073,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
10739,ADNI3,6375,y2,116211,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10814,ADNI3,6252,y2,119133,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10928,ADNI3,6419,y2,124361,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10992,ADNI3,6909,sc,128236,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,3.0,NaN
11082,ADNI3,6362,y2,132618,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,NaN
11100,ADNI3,6949,sc,133457,NaN,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,NaN


In [153]:
gds[gds['GDTOTAL'].isna()]

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE


In [154]:
# Can we infer VISCODE2 from the exam date?
gds[gds['RID'] == 6252]

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
9332,ADNI3,6252,sc,32054,sc,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2018-02-28
10322,ADNI3,6252,y1,96008,m12,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2019-05-06
10814,ADNI3,6252,y2,119133,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN


In [155]:
gds.at[10418, 'VISCODE2'] = gds['VISCODE']
gds.at[10992, 'VISCODE2'] = gds['VISCODE']
gds.at[11100, 'VISCODE2'] = gds['VISCODE']

In [156]:
gds[gds['VISCODE2'].isna()]

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
10587,ADNI3,6220,y1,109073,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
10739,ADNI3,6375,y2,116211,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10814,ADNI3,6252,y2,119133,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10928,ADNI3,6419,y2,124361,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
11082,ADNI3,6362,y2,132618,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,NaN


In [157]:
# Remove participants who failed screening (VISCODE = 'f')
gds = gds[gds.VISCODE != 'f']
gds

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
0,ADNI1,2,sc,6,sc,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2005-08-17
1,ADNI1,3,sc,8,sc,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2005-08-18
2,ADNI1,4,sc,10,sc,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,3.0,2005-08-18
3,ADNI1,5,sc,12,sc,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2005-08-23
4,ADNI1,7,sc,14,sc,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2005-08-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11187,ADNI3,6349,y2,138306,m36,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2021-07-20
11188,ADNI3,6402,y3,138290,m36,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2021-07-20
11189,ADNI3,2333,y3,138518,m120,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2021-07-26
11190,ADNI3,4598,y4,138480,m108,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2021-07-20


In [158]:
gds.isnull().sum()

Phase        0
RID          0
VISCODE      0
ID           0
VISCODE2     5
GDSATIS      0
GDDROP       0
GDEMPTY      0
GDBORED      0
GDSPIRIT     0
GDAFRAID     0
GDHAPPY      0
GDHELP       0
GDHOME       0
GDMEMORY     0
GDALIVE      0
GDWORTH      0
GDENERGY     0
GDHOPE       0
GDBETTER     0
GDTOTAL      0
EXAMDATE    12
dtype: int64

In [159]:
gds[gds['VISCODE2'].isna()]

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
10587,ADNI3,6220,y1,109073,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
10739,ADNI3,6375,y2,116211,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10814,ADNI3,6252,y2,119133,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
10928,ADNI3,6419,y2,124361,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
11082,ADNI3,6362,y2,132618,NaN,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,NaN


In [160]:
gds.sort_values(by= ['Phase', 'VISCODE', 'RID'])

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
739,ADNI1,3,m12,1496,m12,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2006-09-12
928,ADNI1,4,m12,1874,m12,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,2006-11-14
724,ADNI1,5,m12,1466,m12,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2006-09-05
947,ADNI1,6,m12,1912,m12,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2006-11-20
815,ADNI1,7,m12,1648,m12,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,2006-10-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3703,ADNIGO,2402,sc,1232,sc,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,5.0,2011-08-24
3705,ADNIGO,2403,sc,1236,sc,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2011-08-29
3715,ADNIGO,2405,sc,1256,sc,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,5.0,2011-09-19
3728,ADNIGO,2406,sc,1282,sc,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,2011-10-31


In [161]:
gds.iloc[10580:10590]

,Phase,RID,VISCODE,ID,VISCODE2,GDSATIS,GDDROP,GDEMPTY,GDBORED,GDSPIRIT,...,GDHELP,GDHOME,GDMEMORY,GDALIVE,GDWORTH,GDENERGY,GDHOPE,GDBETTER,GDTOTAL,EXAMDATE
10944,ADNI3,6801,y1,125160,m12,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,9.0,2020-11-18
10945,ADNI3,4706,y3,125200,m108,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,3.0,2020-12-16
10946,ADNI3,6821,y1,125224,m12,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2020-12-18
10947,ADNI3,6498,y2,125337,m24,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,2020-09-30
10948,ADNI3,5132,y3,125356,m84,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2020-09-28
10949,ADNI3,6622,y2,125416,m24,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2020-12-02
10950,ADNI3,6840,y1,125432,m12,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,2020-12-21
10951,ADNI3,6784,y1,125520,m12,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2020-08-17
10952,ADNI3,6524,y2,125564,m24,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,2020-12-10
10953,ADNI3,4520,y3,125579,m108,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2020-11-12


In [162]:
gds = gds.dropna()

For the five missing VISCODE2 exam progression is still possible to deduce though VISCODE. It is mostly their last visit in the ADNI3 Phase.

In [163]:
# How many entries did we lose to cleaning?
gds_finish_entries = gds.shape[0]
dropped_entries = gds_start_entries - gds_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/gds_start_entries*100, 2), '% removed')

556
4.89 % removed


In [164]:
# Writing df to csv.
df = gds
df.to_csv('../../data/tabular_data/clean_GDSCALE.csv', index=False)

### Mini Mental State Examination (MMSE)

The Mini-Mental State Exam (MMSE) evaluates orientation, attention, memory, language and visual-spatial skills. It is fine-tuned for population over 55 years of age and is used as a test of cognitive function among the elderly.

    
[TEST EXAMPLE link](https://oxfordmedicaleducation.com/wp-content/uploads/2016/10/MMSE-printable-mini-mental-state-examination.pdf)

1. **Orientation** (10 points)
2. **Registration** (3 points) 
3. **Attention & calculation** (5 points)
4. **Recall** (3 points) 
5. **Language** (8 points) (Word list)
6. **Copying by drawing** (1 point)

The MMSE asks questions to ascertain cognitive status. Responses are scored as following:

0 = incorrect    
1 = correct   
6 = item administered, participant does not answer    
9 = test item not administered, unknown   

**MMSE (0–30):**  

**24-30 no cognitive impairment,  
18-23 = mild impairment,  
0-17 = severe impairment/dementia**

The MMSE does not test executive brain functions or frontal lobe functions including judgment, planning or impulsivity.
However, score <21 indicates increased odds of dementia, while score >25 indicates decreased odds of dementia.

<u>Sources:<u>
- https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/GetPdf.cgi?id=phd001525.1
- https://oxfordmedicaleducation.com/geriatrics/mini-mental-state-examination-mmse/
- http://www.cba.org/CBA/cle/PDF/ELD13_paper_shulman.pdf
- http://www.heartinstitutehd.com/Misc/Forms/MMSE.1276128605.pdf


In [165]:
# MMSE data dictionary, for information about the abbreviations in column names.
mmse_dict = pd.read_csv('../../data/tabular_data/MMSE_dict.csv', header=None)
mmse_dict.style.set_properties(**{'text-align': 'left'}).hide_index()

0,1
Phase,Phase
ID,Record ID
RID,Participant roster ID
SITEID,Site ID
VISCODE,Visit code
VISCODE2,Translated visit code: longitudinal progression
USERDATE,Date record created
USERDATE2,Date record last updated
EXAMDATE,Examination Date
MMDATE,1. What is today's date?1=Correct; 2=Incorrect


Columns:   
    
1. **Orientation (10 points)**: 'MMDATE', 'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA', 'MMSTATE',
2. **Registration** (3 points): 'MMBALL', 'MMFLAG', 'MMTREE', 'MMTRIALS'
3. **Attention & calculation** (5 points): 'MMD','MML', 'MMR', 'MMO', 'MMW'
4. **Recall** (3 points): 'MMBALLDL', 'MMFLAGDL', 'MMTREEDL'
5. **Language** (8 points): 'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 'MMONFLR', 'MMREAD', 'MMWRITE', 'WORLDSCORE'
6. **Copying by drawing** (1 point): 'MMDRAW'   

As we understand from the protocol, the columns after 'MMSCORE' are intended for testers' notes to make it easier for them to calculate some other scores, like WORLDSCORE (score obtained by the participant for each correct letter in 'WORLD' backwards.)   

One task is to spell the word WORLD backwards.
Columns MMLTR1-7 mark the letter that needed to be indicated for the participant (D, O, R ...).    

The 3 words used to test Registration and Recall (Ball/Apple, Flag/Penny, Tree/Table) are scored for correctness in columns MMBALL, MMFLAG, MMTREE (1=correct, 2=incorrect) and similarly in the 3 columns for recall (eg. MMBALL**DL**)
Columns WORD1/WORD1DL -  WORD3/WORD3DL are just translated entries for MMBALL/MMBALLDL, where instead of 1 and 2 values, the entries are 1 point for correct, 0 points for incorrect answer, to make it easier for the tester to sum up the total -> redundant columns? Or added due to changes in protocol?

In [166]:
cols = mmse_dict[0].tolist()
# cols

In [167]:
mmse = pd.read_csv('../../data/tabular_data/MMSE.csv')

# Change hidden missing values.
mmse = mmse.replace('-1', np.NaN)   
mmse = mmse.replace(-1, np.NaN) 
mmse = mmse.replace(-4, np.NaN)
mmse = mmse.replace('-4', np.NaN)
mmse = mmse.replace('-', np.NaN)

C:\Users\kfatur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (8,58,59,60,61,62,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [168]:
# mmse.info()

In [169]:
mmse_start_entries = mmse.shape[0]
mmse.shape

(12353, 74)

In [170]:
# Drop the redundant columns. 
mmse = mmse.drop(['USERDATE', 'USERDATE2'], axis=1)

In [171]:
mmse.isnull().sum()

Phase               0
ID                  0
RID                 0
SITEID              0
VISCODE             0
VISCODE2           16
EXAMDATE         8071
MMDATE             55
MMDATECM          104
MMYEAR             54
MMYEARCM           95
MMMONTH            56
MMMNTHCM          101
MMDAY              54
MMDAYCM            92
MMSEASON           54
MMSESNCM           91
MMHOSPIT           54
MMHOSPCM          108
MMFLOOR            54
MMFLRCM            93
MMCITY             54
MMCITYCM           91
MMAREA             54
MMAREACM          117
MMSTATE            54
MMSTCM             83
MMRECALL         7288
MMBALL           2319
MMFLAG           2319
MMTREE           2319
MMTRIALS         2329
MMD              2323
MMDLTR           2369
MML              2323
MMLLTR           2396
MMR              2323
MMRLTR           2446
MMO              2323
MMOLTR           2461
MMW              2323
MMWLTR           2713
MMBALLDL         2320
MMFLAGDL         2320
MMTREEDL         2320
MMWATCH   

In [172]:
mmse['MMLTR1'].unique()

array([nan, 'D', 'd', 'E', 'W', 'w', '0', 'l', 'O', 'L', 'o', 'y', 'P'],
      dtype=object)

In [173]:
mmse['MMLTR1'] = mmse['MMLTR1'].str.upper()
mmse['MMLTR2'] = mmse['MMLTR2'].str.upper()
mmse['MMLTR3'] = mmse['MMLTR3'].str.upper()
mmse['MMLTR4'] = mmse['MMLTR4'].str.upper()
mmse['MMLTR5'] = mmse['MMLTR5'].str.upper()
mmse['MMLTR6'] = mmse['MMLTR7'].str.upper()
# mmse['MMLTR1'].unique()

In [174]:
mmse['MMLTR7'].unique()

array([nan, '_'], dtype=object)

In [175]:
# Drop the column with no valid entries.
mmse = mmse.drop(['MMLTR7'], axis=1)

In [176]:
# MMSCORE is up to 30, no negative values.
print(mmse['MMSCORE'].max())
print(mmse['MMSCORE'].min())

30.0
0.0


In [177]:
mmse[mmse['MMSCORE'].isna()].head(20)

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,EXAMDATE,MMDATE,MMDATECM,MMYEAR,...,MMLTR6,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORDLIST,WORLDSCORE,update_stamp
545,ADNI1,1108,44,10,m06,m06,2006-05-19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-05-26 00:00:00.0
977,ADNI1,1972,182,18,m06,m06,2006-09-11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-10-05 00:00:00.0
3051,ADNI1,6124,726,19,m12,m12,2007-11-14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-05-05 00:00:00.0
3572,ADNI1,7166,1024,19,m24,m24,2008-10-21,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-23 00:00:00.0
3680,ADNI1,8572,1205,109,m24,m24,2009-03-16,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-31 00:00:00.0
3817,ADNI1,7658,1402,41,m24,m24,2009-05-28,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-06-04 00:00:00.0
3878,ADNI1,7780,1157,86,m24,m24,2009-01-24,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-23 00:00:00.0
3916,ADNI1,8570,1393,12,m48,m48,2011-04-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-14 00:00:00.0
4100,ADNI1,8226,1088,115,m36,m36,2009-12-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-02-24 00:00:00.0
4102,ADNI1,8080,1070,101,m36,m36,2009-12-10,1.0,10,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-12-05 00:00:00.0


The entries with the missing MMSE Score mostly have NaN values in other cells, so we cannot calculate it by ourselves. It is also difficult to infer, why the values are missing, since sometimes (like with RID = 722), the test starts correctly, but is then prematurely ended with no values entered. There is no ground to conclude that the test was interrupted because the participant could not complete it - some of the participants involved only exhibit mild cognitive impairment and should be able to follow the test. Since it is unclear, the entries will be dropped.

In [178]:
mmse = mmse.dropna(subset=['MMSCORE'])

Also, only a smaller number of entries have information on which word list ("ball-flag-tree" or "apple-penny-table") they used.   
From the Protocol we know, that they mostly used the "ball-list" and we don't have sufficient data to compare the performance between the two lists (besides, there should be no significant difference, since the lists are construed based on the frequency of word use.) The column won't be included.

In [179]:
# Subset of columns we are interested in, we don't need verbatim responses (eg. 'MMDATECM'), just correct/incorrect information ('MMDATE').
# MMLTR1-7 columns note which letter was indicated in the word 'WORLD' backwards > 'D-L-R-O-W'
# Score for the 'WORLD' backwards spelling task: 'WORLDSCORE'
mmse = mmse[['Phase', 'ID', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE',
            'MMDATE', 'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 
            'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA', 'MMSTATE',
            'MMBALL', 'MMFLAG', 'MMTREE', 'MMTRIALS', 'MMD','MML',
             'MMR', 'MMO', 'MMW', 'MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 
             'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 
             'MMONFLR', 'MMREAD', 'MMWRITE', 'MMDRAW', 'WORD1', 'WORD1DL',
             'WORD2', 'WORD2DL', 'WORD3', 'WORD3DL', 'WORLDSCORE', 'MMSCORE']]

pd.set_option('display.max_columns', 50)

In [180]:
mmse.head()

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE
0,ADNI1,10,2,sc,sc,2005-08-17,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0
1,ADNI1,12,1,f,f,2005-08-18,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0
2,ADNI1,14,3,sc,sc,2005-08-18,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
3,ADNI1,16,4,sc,sc,2005-08-18,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0
4,ADNI1,18,5,sc,sc,2005-08-23,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0


In [181]:
mmse.isnull().sum()

Phase             0
ID                0
RID               0
VISCODE           0
VISCODE2         12
EXAMDATE       8022
MMDATE            0
MMYEAR            0
MMMONTH           1
MMDAY             0
MMSEASON          0
MMHOSPIT          0
MMFLOOR           0
MMCITY            0
MMAREA            0
MMSTATE           0
MMBALL         2263
MMFLAG         2263
MMTREE         2263
MMTRIALS       2274
MMD            2264
MML            2264
MMR            2264
MMO            2264
MMW            2264
MMBALLDL       2263
MMFLAGDL       2263
MMTREEDL       2263
MMWATCH           1
MMPENCIL          1
MMREPEAT          1
MMHAND            1
MMFOLD            1
MMONFLR           1
MMREAD            0
MMWRITE           0
MMDRAW            1
WORD1         10029
WORD1DL       10028
WORD2         10029
WORD2DL       10027
WORD3         10029
WORD3DL       10027
WORLDSCORE    10038
MMSCORE           0
dtype: int64

In [182]:
print (2263 + 10038)
print (2263 + 10029)

12301
12292


The curious thing is, that summing up the NaNs in columns MMBALL to MMTREEDL and WORLDSSCORE to WORD3DL comes very close to the number of all entries.   

It is possible that this is the same information, just encoded differently thoughout the Phases, meaning, in ADNI1 and ADNI2 they would enter the results for word list task in the MMBALL, MMFLAG, MMTREE columns, but in ADNI3 they would change the protocol and enter the values under WORD1, WORD2, WORD3?    

We should check that, but first we will fix minor things, like visit codes and dates.

We'll drop the individual entries with completely or partially missing test results.

In [183]:
mmse = mmse.dropna(subset=['MMMONTH'])

In [184]:
mmse = mmse.dropna(subset=['MMWATCH'])

In [185]:
mmse = mmse.dropna(subset=['MMDRAW'])

In [186]:
# Filter out those, who failed screening, VISCODE = 'f'
mmse = mmse[mmse.VISCODE != 'f']

In [187]:
mmse.shape

(11880, 45)

In [188]:
mmse[mmse['VISCODE2'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE
9900,ADNI2,9870,5127,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0
9901,ADNI2,9872,5083,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0
9999,ADNI2,10070,4552,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
10010,ADNI2,10092,4198,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0
10021,ADNI2,10114,5200,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0
10060,ADNI2,10192,4028,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
10061,ADNI2,10194,5126,tau,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
12059,ADNI3,121665,6375,y2,NaN,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,28.0
12168,ADNI3,128234,6909,sc,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0
12180,ADNI3,128826,6688,y2,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0


In [189]:
# 'tau' indicates that tau samples were taken, viscode2 = viscode.
# The remaining missing codes will be fixed with the help of EXAMDATE.
mmse.at[9900, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[9901, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[9999, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[10010, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[10021, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[10060, 'VISCODE2'] = mmse['VISCODE']
mmse.at[10061, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[12168, 'VISCODE2'] = mmse['VISCODE'] 
mmse.at[12263, 'VISCODE2'] = mmse['VISCODE'] 
mmse[mmse['VISCODE2'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE
12059,ADNI3,121665,6375,y2,NaN,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,28.0
12180,ADNI3,128826,6688,y2,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0
12200,ADNI3,129924,6839,y1,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,20.0


In [190]:
# Merge exam dates with the 'examdate' df created from the registry above.
mmse = pd.merge(mmse, examdate, how='left', on=['Phase', 'RID', 'VISCODE'])

We fix almost all examdates, which means we'll be probably able to infer the missing VISCODE2 as well. First, we will compare the old date column and the merged date column and try to preserve or infer as much information as possible.

In [191]:
mmse[mmse['EXAMDATE_y'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
1748,ADNI1,4326,304,m06,m06,2007-02-15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN
3589,ADNI1,8028,1430,m36,m36,2009-10-01,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN
4502,ADNIGO,1352,830,m60,m60,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN
9044,ADNI2,9024,5256,v11,m12,NaN,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN
9387,ADNI2,9730,4555,v31,m36,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN
11591,ADNI3,121665,6375,y2,NaN,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,28.0,NaN
11696,ADNI3,128234,6909,sc,0 sc 2 sc 3 sc 4 s...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,NaN
11708,ADNI3,128826,6688,y2,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,NaN
11791,ADNI3,133453,6949,sc,0 sc 2 sc 3 sc 4 s...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,29.0,NaN


In [192]:
# Let's try to fix date manually, where the date is available from the original column.
mmse.at[1748, 'EXAMDATE_y'] = mmse['EXAMDATE_x']
mmse.at[3589, 'EXAMDATE_y'] = mmse['EXAMDATE_x']

In [193]:
# Let's try to fix date using the visit codes and previous dates of examinations.
mmse[mmse['RID'] == 830]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
547,ADNI1,1636,830,sc,sc,2006-08-17,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2006-08-17
1357,ADNI1,3542,830,m06,m06,2007-04-03,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2007-04-03
1952,ADNI1,4736,830,m12,m12,2007-09-17,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2007-09-17
2517,ADNI1,5868,830,m18,m18,2008-03-24,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2008-03-24
2971,ADNI1,6778,830,m24,m24,2008-09-25,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,2008-09-25
3543,ADNI1,7936,830,m36,m36,2009-09-21,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2009-09-21
4029,ADNIGO,396,830,m48,m48,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,2010-09-23
4502,ADNIGO,1352,830,m60,m60,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN
4885,ADNI2,668,830,v06,m60,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,2011-09-22


In [194]:
# Previous exam for RID = 830 at VISCODE2 'm48' was on 2010-09-23, so the exam at 'm60' happened approximately 12 months later.
# Date format: yyyy-mm-dd
mmse.at[4909, 'EXAMDATE_y'] = '2011-09-22'

In [195]:
mmse[mmse['RID'] == 5256]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
7401,ADNI2,5714,5256,v01,sc,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,2013-07-02
8092,ADNI2,7106,5256,v05,m06,NaN,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2014-02-06
9044,ADNI2,9024,5256,v11,m12,NaN,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN
9107,ADNI2,9152,5256,v21,m24,NaN,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2015-09-08


In [196]:
# Exam at month 6 on 8.9.2014, exam at month 12:
mmse.at[9044, 'EXAMDATE_y'] = '2015-03-08'

In [197]:
mmse[mmse['RID'] == 4555]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
5383,ADNI2,1672,4555,v01,sc,NaN,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2012-02-17
6512,ADNI2,3932,4555,v05,m06,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,2012-11-07
7224,ADNI2,5358,4555,v11,m12,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,2013-05-14
8234,ADNI2,7392,4555,v21,m24,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,2014-03-20
9387,ADNI2,9730,4555,v31,m36,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN
9592,ADNI2,10144,4555,v41,m48,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,2016-04-22


In [198]:
mmse.at[9387, 'EXAMDATE_y'] = '2015-04-22'

In [199]:
mmse[mmse['RID'] == 6375]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
10285,ADNI3,42143,6375,sc,sc,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,27.0,2018-05-10
11591,ADNI3,121665,6375,y2,NaN,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,28.0,NaN


In [200]:
# VISCODE = 'y2' means ADNI3 Year 2 Visit, so 24 months from screening ('sc') visit.
mmse.at[11591, 'EXAMDATE_y'] = '2020-05-10'
mmse.at[11591, 'VISCODE2'] = 'm24'

In [201]:
mmse[mmse['RID'] == 6688]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
10763,ADNI3,75674,6688,sc,sc,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,5.0,27.0,2019-02-07
11427,ADNI3,110946,6688,y1,m12,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,29.0,2020-03-05
11708,ADNI3,128826,6688,y2,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,NaN


In [202]:
# VISCODE = 'y2' means ADNI3 Year 2 Visit, so 12 months from 'y1' visit in the table above.
mmse.at[11708, 'EXAMDATE_y'] = '2021-03-05'
mmse.at[11708, 'VISCODE2'] = 'm24'

In [203]:
mmse[mmse['RID'] == 6839]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
11293,ADNI3,103935,6839,sc,sc,NaN,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,23.0,2019-12-03
11728,ADNI3,129924,6839,y1,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,20.0,2021-03-25


RID 6839 got a very late first year examination (3 months later as appropriate), but we will consider this type of exam still as m12, since it was a yearly exam according to VISCODE.

In [204]:
mmse.at[11728, 'VISCODE2'] = 'm12'

In [205]:
mmse[mmse['RID'] == 6909]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
11696,ADNI3,128234,6909,sc,0 sc 2 sc 3 sc 4 s...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,NaN


In [206]:
mmse[mmse['RID'] == 6949]

,Phase,ID,RID,VISCODE,VISCODE2,EXAMDATE_x,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE_y
11791,ADNI3,133453,6949,sc,0 sc 2 sc 3 sc 4 s...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,29.0,NaN


The participants with RIDs 6909 and 6949, who only completed 1 test at one examination and no date was entered, will be removed.

In [207]:
# Drop the entry with missing dates.
mmse = mmse.dropna(subset=['EXAMDATE_y'])

In [208]:
# Drop the old date column. 
mmse = mmse.drop(['EXAMDATE_x'], axis=1)

# Rename the complete EXAMDATE_y.
mmse.rename({'EXAMDATE_y': 'EXAMDATE'}, axis=1, inplace=True)

In [209]:
# Is participant 830 now complete, can we sort by dates?
mmse[mmse['RID'] == 830].sort_values(by='EXAMDATE')

,Phase,ID,RID,VISCODE,VISCODE2,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE
547,ADNI1,1636,830,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2006-08-17
1357,ADNI1,3542,830,m06,m06,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2007-04-03
1952,ADNI1,4736,830,m12,m12,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2007-09-17
2517,ADNI1,5868,830,m18,m18,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2008-03-24
2971,ADNI1,6778,830,m24,m24,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,2008-09-25
3543,ADNI1,7936,830,m36,m36,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2009-09-21
4029,ADNIGO,396,830,m48,m48,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,2010-09-23
4885,ADNI2,668,830,v06,m60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,2011-09-22


In [210]:
# mmse.isnull().sum()

In [211]:
mmse[mmse['MMTRIALS'].isna()].sort_values(by = 'MMSCORE')

,Phase,ID,RID,VISCODE,VISCODE2,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMTRIALS,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE
11355,ADNI3,107529,6100,y2,m24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2020-01-27
11079,ADNI3,92460,6231,y1,m12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2019-07-23
11188,ADNI3,88324,6756,sc,sc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,2019-07-03
11812,ADNI3,134879,6661,y2,m24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,2021-04-08
10677,ADNI3,69739,6638,sc,sc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,2018-11-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10301,ADNI3,43087,6386,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2018-05-21
10298,ADNI3,43189,6354,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2018-05-04
10295,ADNI3,42905,6380,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2018-05-16
10309,ADNI3,44039,6392,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2018-05-23


In [212]:
mmse['MMTRIALS'].unique()

array([ 1.,  2.,  4.,  3.,  5.,  6., nan])

Missing values in the number of trials are probably related to the changed manner of entering the test results in ADNI3 and will remain missing for those 2250+ entries. Since NEUROBAT test is more sensitive to trial and error we will dismiss this column entirely in MMSE due to the discrepancy in data quality and low information value.

In [213]:
mmse = mmse.drop(['MMTRIALS'], axis=1)

Our observation is also, that the columns MMBALL, MMFLAG, MMTREE mark the correct answer for the words 'ball', 'tree', 'flag'. In ADNI3 this correctness was marked under WORD1, WORD2, WORD3, because there are 2 lists with 3 words (ball, flag, tree / apple, penny, table) and they wanted to record the correct answer without specifying the list. The idea is to change the lists, so that the participants would not remember it from previous testing sessions, but they ended up using mostly the first list. (This is **not** very clearly marked in the [Protocol](https://adni.loni.usc.edu/wp-content/uploads/2012/10/ADNI3-Procedures-Manual_v3.0_20170627.pdf).)    

Spelling 'WORLD' backwards is recorded in the columns MMD, MML, MMR, MMO and MMW for earlier phases, but then in ADNI3 the sum of correct letters is entered just as sum in one column, 'WORLDSCORE.   

We need to unify these columns, since they basically hold the same kind of information.

The problem is also, that earlier, the wrong answer was encoded with '2', making it harder for us to just sum up the columns. We will encode wrong answer as '0' from now on.

In [214]:
mmse['MMBALL'].unique()

array([ 1.,  2., nan])

In [215]:
# Wrong answers will be encoded with 0 instead of 2, it makes it easier to sum the columns.
mmse.loc[(mmse.MMBALL > 1), 'MMBALL'] = 0
mmse.loc[(mmse.MMFLAG > 1), 'MMFLAG'] = 0
mmse.loc[(mmse.MMTREE > 1), 'MMTREE'] = 0
mmse.loc[(mmse.MMBALLDL > 1), 'MMBALLDL'] = 0
mmse.loc[(mmse.MMFLAGDL > 1), 'MMFLAGDL'] = 0
mmse.loc[(mmse.MMTREEDL > 1), 'MMTREEDL'] = 0

In [216]:
mmse.loc[(mmse.MMDATE > 1), 'MMDATE'] = 0
mmse.loc[(mmse.MMYEAR > 1), 'MMYEAR'] = 0
mmse.loc[(mmse.MMMONTH > 1), 'MMMONTH'] = 0
mmse.loc[(mmse.MMDAY > 1), 'MMDAY'] = 0
mmse.loc[(mmse.MMSEASON > 1), 'MMSEASON'] = 0
mmse.loc[(mmse.MMHOSPIT  > 1), 'MMHOSPIT'] = 0
mmse.loc[(mmse.MMFLOOR > 1), 'MMFLOOR'] = 0
mmse.loc[(mmse.MMCITY > 1), 'MMCITY'] = 0
mmse.loc[(mmse.MMAREA > 1), 'MMAREA'] = 0
mmse.loc[(mmse.MMSTATE > 1), 'MMSTATE'] = 0

In [217]:
mmse['WORD1'].unique()

array([nan,  1.,  0.])

In [218]:
mmse['MMWATCH'].unique()

array([1., 2., 0.])

In [219]:
mmse.loc[(mmse.MMWATCH > 1), 'MMWATCH'] = 0
mmse.loc[(mmse.MMPENCIL > 1), 'MMPENCIL'] = 0
mmse.loc[(mmse.MMREPEAT > 1), 'MMREPEAT'] = 0
mmse.loc[(mmse.MMHAND > 1), 'MMHAND'] = 0
mmse.loc[(mmse.MMFOLD > 1), 'MMFOLD'] = 0
mmse.loc[(mmse.MMONFLR > 1), 'MMONFLR'] = 0
mmse.loc[(mmse.MMREAD > 1), 'MMREAD'] = 0
mmse.loc[(mmse.MMWRITE > 1), 'MMWRITE'] = 0
mmse.loc[(mmse.MMDRAW > 1), 'MMDRAW'] = 0

In [220]:
# How is letter W in spelling WORLD assessed? Correct -1, incorrect - 2?
mmse['MMW'].unique()

array([ 1.,  2., nan])

In [221]:
# We need to change 2 to 0 to calculate 'WORLDSCORE' for missing entries.
mmse.loc[(mmse.MMD > 1), 'MMD'] = 0
mmse.loc[(mmse.MML > 1), 'MML'] = 0
mmse.loc[(mmse.MMR > 1), 'MMR'] = 0
mmse.loc[(mmse.MMO > 1), 'MMO'] = 0
mmse.loc[(mmse.MMW > 1), 'MMW'] = 0

In [222]:
pd.set_option('display.max_columns', 50)
mmse.head(10)

,Phase,ID,RID,VISCODE,VISCODE2,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMD,MML,MMR,MMO,MMW,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE
0,ADNI1,10,2,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2005-08-17
1,ADNI1,14,3,sc,sc,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,2005-08-18
2,ADNI1,16,4,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,2005-08-18
3,ADNI1,18,5,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2005-08-23
4,ADNI1,22,7,sc,sc,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,2005-08-25
5,ADNI1,24,8,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2005-09-13
6,ADNI1,30,14,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2005-09-12
7,ADNI1,32,16,sc,sc,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,2005-09-13
8,ADNI1,34,15,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2005-09-15
9,ADNI1,38,19,sc,sc,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,2005-09-20


In [223]:
# Calculate missing WORLDSCORE from columns counting correctness of each letter.
columns_list = ['MMD', 'MML', 'MMR', 'MMO', 'MMW']
mmse['WORLDSCORE_temp'] = mmse[columns_list].sum(axis=1)

In [224]:
# Fill the NaN in 'WORLDSCORE' from temp column.
mmse['WORLDSCORE'].fillna(mmse['WORLDSCORE_temp'], inplace=True)

In [225]:
# Drop individual columns for spelling D-L-R-O-W, drop temp column
mmse = mmse.drop(columns = columns_list, axis=1)
mmse = mmse.drop(['WORLDSCORE_temp'], axis=1)

In [226]:
mmse.head()

,Phase,ID,RID,VISCODE,VISCODE2,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE
0,ADNI1,10,2,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,28.0,2005-08-17
1,ADNI1,14,3,sc,sc,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,20.0,2005-08-18
2,ADNI1,16,4,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,27.0,2005-08-18
3,ADNI1,18,5,sc,sc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,29.0,2005-08-23
4,ADNI1,22,7,sc,sc,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,20.0,2005-08-25


In [227]:
# Where values for MMBALL, MMFLAG etc. are missing, these values are entered in WORD1, WORD2, WORD3 ...
mmse[mmse['MMBALL'].isna()]

,Phase,ID,RID,VISCODE,VISCODE2,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMBALL,MMFLAG,MMTREE,MMBALLDL,MMFLAGDL,MMTREEDL,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORLDSCORE,MMSCORE,EXAMDATE
9619,ADNI3,938,1078,init,m120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,14.0,2017-02-22
9620,ADNI3,2013,120,init,m132,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,29.0,2017-03-29
9621,ADNI3,1486,1261,init,m120,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2017-03-14
9622,ADNI3,1175,1280,init,m120,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2017-03-02
9623,ADNI3,490,2183,init,m72,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,28.0,2017-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,ADNI3,138287,6402,y3,m36,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,5.0,27.0,2021-07-20
11876,ADNI3,138431,6351,y2,m24,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,29.0,2020-08-05
11877,ADNI3,138512,2333,y3,m120,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,24.0,2021-07-26
11878,ADNI3,138474,4598,y4,m108,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,30.0,2021-07-20


In [228]:
# Fill missing values in WORD1 et co. from columns MMBALL et co. (the all test the same thing - how many of the 3 words did the participant remember).
# mmse['WORD1'].fillna(mmse['MMBALL'], inplace=True)
mmse['WORD2'].fillna(mmse['MMBALL'], inplace=True)
mmse['WORD1'].fillna(mmse['MMFLAG'], inplace=True)
mmse['WORD3'].fillna(mmse['MMTREE'], inplace=True)
mmse['WORD1DL'].fillna(mmse['MMBALLDL'], inplace=True)
mmse['WORD2DL'].fillna(mmse['MMFLAGDL'], inplace=True)
mmse['WORD3DL'].fillna(mmse['MMTREEDL'], inplace=True)

In [229]:
# Drop word columns from where the information was extracted to WORD1, WORD2, WORD3 ...
col_list = ['MMBALL', 'MMFLAG', 'MMTREE', 'MMBALLDL', 'MMFLAGDL', 'MMTREEDL']
mmse = mmse.drop(columns = col_list, axis=1)

In [230]:
# mmse.isnull().sum()

In [231]:
mmse = mmse.dropna()

New columns:

**Orientation (10 points):**
'MMDATE', 'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA', 'MMSTATE'
**Registration** (3 points): 'WORD1', 'WORD2', 'WORD3'  
**Attention & calculation** (5 points): 'WORLDSCORE'   
**Recall** (3 points): 'WORD1DL', 'WORD2DL', 'WORD3DL'  
**Language** (8 points): 'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 'MMONFLR', 'MMREAD', 'MMWRITE'   
**Copying by drawing** (1 point): 'MMDRAW'


In [232]:
mmse.shape

(11876, 33)

In [233]:
# How many entries did we lose to cleaning?
mmse_finish_entries = mmse.shape[0]
dropped_entries = mmse_start_entries - mmse_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/mmse_start_entries*100, 2), '% removed')

477
3.86 % removed


In [234]:
# Writing df to csv.
df = mmse
df.to_csv('../../data/tabular_data/clean_MMSE.csv', index=False)

### Montreal Cognitive Assessment (MoCA) 

The Montreal Cognitive Assessment (MoCA) is a rapid screening instrument to assess mild cognitive dysfunction. It covers different cognitive domains: 

1. **Visuospatial/Executive**,
2. **Naming**, 
3. **Memory**, 
4. **Attention and calculation**, 
5. **Language**, 
6. **Conceptual thinking**,
7. **Delayed recall**,
8. **Orientation**.

**MoCa Scale (0-30):**  

**18-25 = mild cognitive impairment    
10-17 = moderate cognitive impairment   
< 10 = severe cognitive impairment**

"If the subject has 12 years of education or fewer, a point is added to his/her total score[, but] if a subject scores 30/30, a point is not added if he/she has 12 years of education or less."

"The cut-off score of 18 is usually considered to separate MCI from AD but there is an overlap in the scores since, by definition, AD is determined by the presence of cognitive impairment in addition to loss of autonomy.   

The average MoCA score for MCI is 22 (range 19-25) and the average MoCA score for Mild AD is 16 (11-21)." Source: [MoCa Test Website](https://www.mocatest.org/faq/)

According to the ADNI Procedures Manual, the MOCA total score is not calculated for ADNI.

<u>Sources:<u>
- https://www.mocatest.org/faq/
- https://www.dementia.org/diagnosing-dementia-the-montreal-cognitive-assessment
    
<u>Test samples<u>:
- https://www.parkinsons.va.gov/resources/MOCA-Test-English.pdf
- https://de.scribd.com/document/393713047/Montreal-cognitive-assessment-Basic-English-FINAL-VERSION-4-June-2014-pdf

In [235]:
moca_dict = pd.read_csv('../../data/tabular_data/MOCA_dict.csv', header=None)
moca_dict

,0,1
0,Phase,Phase
1,ID,Record ID
2,RID,Participant roster ID
3,SITEID,Site ID
4,VISCODE,Visit code
5,VISCODE2,Translated visit code: longitudinal progression
6,USERDATE,Date record created
7,USERDATE2,Date record last updated
8,TRAILS,Trails; 1=Correct; 0=Incorrect
9,CUBE,Copy Cube; 1=Correct; 0=Incorrect(ADNI2&3)


In [236]:
cols = moca_dict[0].tolist()
# cols

Columns (see [test example](https://static.seattletimes.com/wp-content/uploads/2018/01/0bb5d4b6-fbe7-11e7-93e1-366632afe6f9-780x1032.jpg) for clarity:
    
   **Visuospatial/executive**: 'TRAILS', 'CUBE', 'CLOCKCON', 'CLOCKNO', 'CLOCKHAN'   (**5** pts)
   
   **Naming**: 'LION', 'RHINO', 'CAMEL'    (**3** pts)    
   
   **Memory**:   (**no points**, counted only for delayed recall)
   
        > Recall of the read words in 1st trial: 'IMMT1W1', 'IMMT1W2', 'IMMT1W3', 'IMMT1W4', 'IMMT1W5'     
        > Recall of the read words in 2nd trial: 'IMMT2W1', 'IMMT2W2', 'IMMT2W3', 'IMMT2W4', 'IMMT2W5'  
        
   **Attention**:    
   
        > Reading digits: 'DIGFOR', 'DIGBACK'    (**2** pts)
        > Reading a list of letters: 'LETTERS'   (**1** pt; no points if no. of errors >= 2)
        > Serial substraction: 'SERIAL1', 'SERIAL2', 'SERIAL3', 'SERIAL4', 'SERIAL5'   
        (**3** pts; if only 2-3 correct substractions: **2** pts, if 1 correct substraction: **1** pt)
        
   **Language**: 'REPEAT1', 'REPEAT2', 'FFLUENCY'    (**2** pts for correct REPEAT, **1** pt if >=11 words listed)
   
   **Abstraction** (similarity between concepts): 'ABSTRAN', 'ABSMEAS'    (**2** pts)
   
   **Delayed recall** (word list from the Memory part): 'DELW1', 'DELW2', 'DELW3', 'DELW4', 'DELW5'  (**5** pts)
   (Points only for uncued recall.)
   
   **Orientation**: 'DATE', 'MONTH', 'YEAR', 'DAY', 'PLACE', 'CITY'      (**6** pts)  

In [237]:
moca = pd.read_csv('../../data/tabular_data/MOCA.csv')
moca.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,...,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA,update_stamp
0,ADNIGO,4,292,24,m48,m48,2010-05-27,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,2012-11-16 09:41:13.0
1,ADNIGO,6,1326,12,m36,m36,2010-05-30,2011-08-04,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,2012-11-16 09:41:13.0
2,ADNIGO,8,376,17,m48,m48,2010-06-02,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2012-11-16 09:41:13.0
3,ADNIGO,10,1117,12,m36,m36,2010-06-27,2011-11-08,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,1.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,2012-11-16 09:41:13.0
4,ADNIGO,12,618,21,m48,m48,2010-06-28,NaN,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2012-11-16 09:41:13.0


In [238]:
moca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Phase         7042 non-null   object 
 1   ID            7042 non-null   int64  
 2   RID           7042 non-null   int64  
 3   SITEID        7042 non-null   int64  
 4   VISCODE       7042 non-null   object 
 5   VISCODE2      7034 non-null   object 
 6   USERDATE      7042 non-null   object 
 7   USERDATE2     2853 non-null   object 
 8   TRAILS        6961 non-null   float64
 9   CUBE          6961 non-null   float64
 10  CLOCKCON      6950 non-null   float64
 11  CLOCKNO       6949 non-null   float64
 12  CLOCKHAN      6947 non-null   float64
 13  LION          6965 non-null   float64
 14  RHINO         6964 non-null   float64
 15  CAMEL         6965 non-null   float64
 16  IMMT1W1       6948 non-null   float64
 17  IMMT1W2       6949 non-null   float64
 18  IMMT1W3       6946 non-null 

In [239]:
# Change hidden missing values.
moca = moca.replace(-1, np.NaN)    
moca = moca.replace(-4, np.NaN)
moca = moca.replace('-1', np.NaN)    
moca = moca.replace('-4', np.NaN)

moca.isnull().sum()

Phase              0
ID                 0
RID                0
SITEID             0
VISCODE            0
VISCODE2           8
USERDATE           0
USERDATE2       4189
TRAILS            81
CUBE              81
CLOCKCON          92
CLOCKNO           93
CLOCKHAN          95
LION              77
RHINO             78
CAMEL             77
IMMT1W1           94
IMMT1W2           93
IMMT1W3           96
IMMT1W4           98
IMMT1W5           88
IMMT2W1          103
IMMT2W2           98
IMMT2W3          100
IMMT2W4          101
IMMT2W5           95
DIGFOR            78
DIGBACK           79
LETTERS           91
SERIAL1          102
SERIAL2          116
SERIAL3          123
SERIAL4          125
SERIAL5          127
REPEAT1           79
REPEAT2           80
FFLUENCY          87
ABSTRAN           92
ABSMEAS           93
DELW1            103
DELW2             98
DELW3            104
DELW4            106
DELW5            102
DATE              86
MONTH             86
YEAR              87
DAY          

In [240]:
moca_start_entries = moca.shape[0]

In [241]:
moca = moca.drop(columns = ['USERDATE', 'USERDATE2', 'update_stamp'])
moca.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
0,ADNIGO,4,292,24,m48,m48,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN
1,ADNIGO,6,1326,12,m36,m36,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN
2,ADNIGO,8,376,17,m48,m48,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
3,ADNIGO,10,1117,12,m36,m36,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,1.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN
4,ADNIGO,12,618,21,m48,m48,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


<a id='moca_score'></a>
**Entries with MOCA score calculated**:
Some participants (1872) actually have their MOCA score entered, while the majority is missing this value. Let's quickly check the ones who do have the score, to see how complete the rows are.

We might want to check these values later, when we use our own formula to calculate the score for all entries and compare with the originally calculated score. [Compare with the calculated MOCA score.](#calculated_score)

In [242]:
moca[~moca['MOCA'].isna()]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
5106,ADNI3,389,74,20,init,m132,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,17.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0
5107,ADNI3,941,1078,27,init,m120,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0
5108,ADNI3,244,2183,20,init,m72,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,10.0,1.0,1.0,3.0,2.0,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,20.0
5109,ADNI3,424,2219,20,init,m72,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,9.0,1.0,1.0,0.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,21.0
5110,ADNI3,230,2245,20,init,m72,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,21.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,ADNI3,138345,6402,6,y3,m36,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,10.0,1.0,1.0,0.0,3.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,22.0
7038,ADNI3,138355,6962,59,bl,bl,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,10.0
7039,ADNI3,138436,6351,13,y2,m24,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0
7040,ADNI3,138516,2333,55,y3,m120,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,2.0,0.0,3.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,15.0


In [243]:
moca[moca['VISCODE2'].isna()]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
6146,ADNI3,84693,6082,49,y1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6374,ADNI3,97598,2068,18,y1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6424,ADNI3,100322,5273,52,y2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6796,ADNI3,121687,6375,4,y2,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,14.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,29.0
6892,ADNI3,127760,6892,12,bl,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,17.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,25.0
6904,ADNI3,128833,6688,50,y2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,11.0,1.0,1.0,3.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0
6922,ADNI3,129944,6839,12,y1,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,12.0
6957,ADNI3,132821,6931,12,bl,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0,1.0,1.0,3.0,3.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0


In [244]:
# Get the index of entries with missing VISCODE2 and NaN values in all other cells and delete them right away.
i = moca[((moca.Phase == 'ADNI3') &( moca.RID == 6082) & (moca.VISCODE == 'y1'))].index
j = moca[((moca.Phase == 'ADNI3') &( moca.RID == 2068) & (moca.VISCODE == 'y1'))].index
k = moca[((moca.Phase == 'ADNI3') &( moca.RID == 5273) & (moca.VISCODE == 'y2'))].index

moca = moca.drop(i)
moca = moca.drop(j)
moca = moca.drop(k)

In [245]:
moca[moca['RID'] == 6375]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
5768,ADNI3,58320,6375,4,bl,bl,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,29.0
6796,ADNI3,121687,6375,4,y2,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,14.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,29.0


In [246]:
moca.at[6796,'VISCODE2'] = 'm24'

In [247]:
moca[moca['RID'] == 6892]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
6892,ADNI3,127760,6892,12,bl,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,17.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,25.0


In [248]:
# What is VISCODE2 for other participants with WISCODE 'bl'?

moca.Phase = pd.Categorical(moca.Phase, 
                      categories=['ADNI1', 'ADNI2', 'ADNIGO', 'ADNI3'],
                      ordered=True)

viscode = moca[moca['VISCODE'] == 'bl']
viscode.sort_values(by = ['Phase', 'VISCODE2']).tail(10)

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
7014,ADNI3,137162,6927,25,bl,bl,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,13.0,1.0,1.0,3.0,3.0,3.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,16.0
7015,ADNI3,137143,6969,24,bl,bl,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,19.0,0.0,0.0,0.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,22.0
7020,ADNI3,137252,6965,20,bl,bl,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,13.0,1.0,0.0,0.0,3.0,0.0,2.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,17.0
7024,ADNI3,137683,6941,12,bl,bl,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,22.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,26.0
7025,ADNI3,137652,6966,34,bl,bl,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,17.0,1.0,0.0,1.0,2.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,25.0
7030,ADNI3,137946,6920,11,bl,bl,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,13.0,1.0,0.0,3.0,3.0,3.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,18.0
7034,ADNI3,138172,6964,55,bl,bl,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,18.0,1.0,0.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,19.0
7038,ADNI3,138355,6962,59,bl,bl,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,10.0
6892,ADNI3,127760,6892,12,bl,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,17.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,25.0
6957,ADNI3,132821,6931,12,bl,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0,1.0,1.0,3.0,3.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0


In [249]:
moca.at[6892,'VISCODE2'] = 'bl'
moca.at[6957,'VISCODE2'] = 'bl'

In [250]:
moca[moca['RID'] == 6688]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
6028,ADNI3,77458,6688,50,bl,bl,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,14.0,1.0,1.0,0.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,22.0
6632,ADNI3,110953,6688,50,y1,m12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,15.0,1.0,1.0,3.0,2.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,23.0
6904,ADNI3,128833,6688,50,y2,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,11.0,1.0,1.0,3.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0


In [251]:
moca.at[6904,'VISCODE2'] = 'm24'

In [252]:
moca[moca['RID'] == 6839]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
6549,ADNI3,106692,6839,12,bl,bl,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,17.0
6922,ADNI3,129944,6839,12,y1,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,12.0


In [253]:
moca.at[6922,'VISCODE2'] = 'm12'

In [254]:
# How do the entries with missing values in 'TRAILS' column look like?
moca[moca['TRAILS'].isna()].head(10)          

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
75,ADNIGO,158,2106,5,bl,bl,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,6.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
104,ADNIGO,216,869,25,m48,m48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,ADNIGO,922,2210,56,m06,m06,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
469,ADNIGO,946,622,47,m60,m60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,ADNI2,184,552,26,v06,m60,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,0.0,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569,ADNI2,190,161,7,v06,m60,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN
688,ADNI2,428,887,17,v06,m60,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
752,ADNI2,556,667,38,v06,m60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
773,ADNI2,598,869,25,v06,m60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,ADNI2,612,1389,51,v06,m48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
# moca.shape

In [256]:
# moca.isnull().sum()

In [257]:
moca[moca['TRAILS'].isna()]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
75,ADNIGO,158,2106,5,bl,bl,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,6.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
104,ADNIGO,216,869,25,m48,m48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,ADNIGO,922,2210,56,m06,m06,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
469,ADNIGO,946,622,47,m60,m60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,ADNI2,184,552,26,v06,m60,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,0.0,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569,ADNI2,190,161,7,v06,m60,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN
688,ADNI2,428,887,17,v06,m60,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
752,ADNI2,556,667,38,v06,m60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
773,ADNI2,598,869,25,v06,m60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,ADNI2,612,1389,51,v06,m48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
moca[moca['RID'] == 887]

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
52,ADNIGO,110,887,17,m48,m48,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
688,ADNI2,428,887,17,v06,m60,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1984,ADNI2,3028,887,17,v11,m72,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3127,ADNI2,5320,887,17,v21,m84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can't be sure why the values in some rows are missing.     
Was the question not asked? Did the participant not reply, or wasn't able to?   

Assuming that the tester sometimes simply did not score a wrong answer, (and hence NaN is actually 0 points), doesn't seem valied, since some rows contain NaN, 0 (incorrect answer) and 1 (correct answer). 

If we check the participant with RID 887 in Phase ADNI2, it seems plausible, that the cognitive state of the patient was so poor, that the tester simply skipped over the more complex tasks. But is it OK to mark the not performed tasks as failed tasks?

Since we are risking a moderately low proportion of entries (about 100/6971 or 1.4%), we will manually compare a couple of entries, especially for RIDs with several exams, and in case they scored poorly overall, we will assume that NaN simply mean 0 points. Just dropping these cases might skew the final results, since we would mainly drop the entries for poor performers.

Where all entries are NaN, the row will be dropped, since without at least 1 entered value we cannot confirm that the test was attempted.

In [259]:
# Delete the rows with NaN in all columns.
cols = moca.iloc[:,6:-1].columns.to_list()
cols
moca = moca.dropna(subset=cols, how = 'all')

In [260]:
# moca.isnull().sum()

In [261]:
moca[moca['IMMT1W1'].isna()].head(10)

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,DIGBACK,LETTERS,SERIAL1,SERIAL2,SERIAL3,SERIAL4,SERIAL5,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,MOCA
688,ADNI2,428,887,17,v06,m60,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1094,ADNI2,1246,4488,5,v03,bl,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,13.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN
1308,ADNI2,1674,1318,20,v06,m60,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2477,ADNI2,4018,1318,20,v11,m72,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,NaN
3309,ADNI2,5684,4282,37,v21,m24,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3406,ADNI2,5878,1057,2,v11,m72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4040,ADNI2,7146,5029,36,v11,m12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4619,ADNI2,8308,8,8,v41,m120,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,12.0,0.0,1.0,2.0,2.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
4937,ADNI2,8952,5202,30,v31,m36,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,22.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN
5138,ADNI3,2337,4254,16,init,m60,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,9.0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,24.0


It is safe to assume that NaN in these cases would be 0, because these participants scored low overall.

In [262]:
moca.shape

(6971, 49)

In [263]:
moca[cols] = moca[cols].fillna(value=0)

In [264]:
# Since MOCA column has a lot of NaN and we know how to calculate the score, we'll make a new one.
moca.drop('MOCA', axis=1, inplace=True)

There are specific rules to score the columns, so first we will prepare the df for summing up the values per row, as it is not a straightforward sum.

In [265]:
# Column 'LETTERS' contains the number of errors and not points, so whenever the value in this column is >= 2, it counts as 0 points.  
moca['LETTERS_points'] = moca[['LETTERS']].apply(lambda x: 0 if x['LETTERS'] >= 2 else 1, axis=1)

In [266]:
# Sum the points in these columns.
moca['SERIAL_sum'] = moca.loc[:,['SERIAL1', 'SERIAL2', 'SERIAL3', 'SERIAL4', 'SERIAL5']].sum(axis=1)

# Columns SERIAL1-5: if the sum of columns is 4+: 3 points, if the sum is 2 or 3: 2 points, for sum 1: 1 point, else 0
# Map sum of columns to points for columns.
serial_points = {5: 3, 4: 3, 3: 2, 2: 2, 1: 1, 0: 0}
moca['SERIAL_points'] = moca['SERIAL_sum'].map(serial_points)

In [267]:
# FFLUENCY: 1 point if the participant lists 11+ words ('FFLUENCY' >= 11 --> 1 point) 
moca['FLUENCY_points'] = moca[['FFLUENCY']].apply(lambda x: 1 if x['FFLUENCY'] >= 11 else 0, axis=1)

In [268]:
moca['DELW1'].unique()

array([0., 3., 2., 1.])

In [269]:
# DELW1-5: 1 point if the participant recalls the word without cue (values 2 and 3 indicate cueing and count as 0 points) 
recall_points = {3: 0, 2: 0, 1: 1, 0: 0}
moca['RECALL1'] = moca['DELW1'].map(recall_points)
moca['RECALL2'] = moca['DELW2'].map(recall_points)
moca['RECALL3'] = moca['DELW3'].map(recall_points)
moca['RECALL4'] = moca['DELW4'].map(recall_points)
moca['RECALL5'] = moca['DELW5'].map(recall_points)

In [270]:
# Are the results as expected?
moca.head(10)

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,...,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,LETTERS_points,SERIAL_sum,SERIAL_points,FLUENCY_points,RECALL1,RECALL2,RECALL3,RECALL4,RECALL5
0,ADNIGO,4,292,24,m48,m48,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,5.0,3,0,0,0,0,0,0
1,ADNIGO,6,1326,12,m36,m36,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,1.0,1,0,0,0,0,0,0
2,ADNIGO,8,376,17,m48,m48,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,5.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1,5.0,3,0,0,0,0,0,0
3,ADNIGO,10,1117,12,m36,m36,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,11.0,1.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0.0,0,1,0,0,0,0,0
4,ADNIGO,12,618,21,m48,m48,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,12.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,5.0,3,1,0,1,0,0,1
5,ADNIGO,14,2002,48,bl,bl,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,30.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,5.0,3,1,0,1,0,1,1
6,ADNIGO,16,2007,12,bl,bl,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,20.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1,5.0,3,1,0,0,0,0,0
7,ADNIGO,18,2010,1,bl,bl,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,18.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,5.0,3,1,1,1,0,0,1
8,ADNIGO,20,622,47,m48,m48,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,10.0,1.0,1.0,2.0,3.0,1.0,3.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1,3.0,2,0,0,0,1,0,0
9,ADNIGO,22,685,1,m48,m48,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,11.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1,5.0,3,1,0,0,0,0,0


In [271]:
moca.drop('SERIAL_sum', axis=1, inplace=True)

In [272]:
# Columns which don't count for final score.  
dont_count = ['IMMT1W1', 
              'IMMT1W2',
              'IMMT1W3',
              'IMMT1W4',
              'IMMT1W5',
              'IMMT2W1',
              'IMMT2W2',
              'IMMT2W3',
              'IMMT2W4',
              'IMMT2W5',
              'LETTERS',
              'SERIAL1',
              'SERIAL2',
              'SERIAL3',
              'SERIAL4',
              'SERIAL5',
              'FFLUENCY', 
              'DELW1', 
              'DELW2', 
              'DELW3', 
              'DELW4', 
              'DELW5']


# Column to calculate MOCA score.
moca['MOCASCORE'] = moca.iloc[:,6:].sum(axis=1) - moca[dont_count].sum(axis=1)

# # Fill the NaN in 'WORLDSCORE' from temp column.
# mmse['WORLDSCORE'].fillna(mmse['WORLDSCORE_temp'], inplace=True)

In [273]:
moca.head(10)

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,TRAILS,CUBE,CLOCKCON,CLOCKNO,CLOCKHAN,LION,RHINO,CAMEL,IMMT1W1,IMMT1W2,IMMT1W3,IMMT1W4,IMMT1W5,IMMT2W1,IMMT2W2,IMMT2W3,IMMT2W4,IMMT2W5,DIGFOR,...,REPEAT1,REPEAT2,FFLUENCY,ABSTRAN,ABSMEAS,DELW1,DELW2,DELW3,DELW4,DELW5,DATE,MONTH,YEAR,DAY,PLACE,CITY,LETTERS_points,SERIAL_points,FLUENCY_points,RECALL1,RECALL2,RECALL3,RECALL4,RECALL5,MOCASCORE
0,ADNIGO,4,292,24,m48,m48,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,3,0,0,0,0,0,0,18.0
1,ADNIGO,6,1326,12,m36,m36,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,1,0,0,0,0,0,0,13.0
2,ADNIGO,8,376,17,m48,m48,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,5.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1,3,0,0,0,0,0,0,21.0
3,ADNIGO,10,1117,12,m36,m36,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,11.0,1.0,0.0,3.0,3.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0,1,0,0,0,0,0,12.0
4,ADNIGO,12,618,21,m48,m48,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,12.0,1.0,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,3,1,0,1,0,0,1,26.0
5,ADNIGO,14,2002,48,bl,bl,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,30.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,3,1,0,1,0,1,1,28.0
6,ADNIGO,16,2007,12,bl,bl,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,20.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1,3,1,0,0,0,0,0,23.0
7,ADNIGO,18,2010,1,bl,bl,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,18.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,3,1,1,1,0,0,1,27.0
8,ADNIGO,20,622,47,m48,m48,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,10.0,1.0,1.0,2.0,3.0,1.0,3.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1,2,0,0,0,1,0,0,21.0
9,ADNIGO,22,685,1,m48,m48,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,11.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1,3,1,0,0,0,0,0,24.0


<a id='calculated_score'></a>
Some participants have had their MOCA score calculated previously, we've checked it right at the beginning. 
[Entries with the original MOCA score](#moca_score).

If we've done the calculation right, the values should match.

In [274]:
# The entry with ID 389 had MOCA score of 24.0
print(moca[moca['ID']==389]['MOCASCORE'])

# The entry with ID 941 had MOCA score of 8.0
print(moca[moca['ID']==941]['MOCASCORE'])

# The entry with ID 138476 had MOCA score of 27.0
print(moca[moca['ID']==138476]['MOCASCORE'])

5106    24.0
Name: MOCASCORE, dtype: float64
5107    8.0
Name: MOCASCORE, dtype: float64
7041    27.0
Name: MOCASCORE, dtype: float64


In [279]:
# Yay, lets's wrap it up.
moca.isnull().sum()

Phase             0
ID                0
RID               0
SITEID            0
VISCODE           0
VISCODE2          0
TRAILS            0
CUBE              0
CLOCKCON          0
CLOCKNO           0
CLOCKHAN          0
LION              0
RHINO             0
CAMEL             0
IMMT1W1           0
IMMT1W2           0
IMMT1W3           0
IMMT1W4           0
IMMT1W5           0
IMMT2W1           0
IMMT2W2           0
IMMT2W3           0
IMMT2W4           0
IMMT2W5           0
DIGFOR            0
DIGBACK           0
LETTERS           0
SERIAL1           0
SERIAL2           0
SERIAL3           0
SERIAL4           0
SERIAL5           0
REPEAT1           0
REPEAT2           0
FFLUENCY          0
ABSTRAN           0
ABSMEAS           0
DELW1             0
DELW2             0
DELW3             0
DELW4             0
DELW5             0
DATE              0
MONTH             0
YEAR              0
DAY               0
PLACE             0
CITY              0
LETTERS_points    0
SERIAL_points     0


We added columns LETTERS_points, SERIAL_points, FLUENCY_points, and columns for recall RECALL1-5 to calculate the MOCASCORE for all participants. We will leave them in the dataframe, since the original columns carry information that is more qualitative in nature (e.g. after how many hints a participant got the answer right) and are less suitable for scoring. However, we want to retain this qualitative value of data, should we explore it later when nuancing types of errors.  

In [276]:
# How many entries did we lose due to cleaning?
moca_finish_entries = moca.shape[0]
dropped_entries = moca_start_entries - moca_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/moca_start_entries*100, 2), '% removed')

71
1.01 % removed


In [277]:
# Writing df to csv.
df = moca
df.to_csv('../../data/tabular_data/clean_MOCA.csv', index=False)

### Neuropsychological Battery Tests (NEUROBAT)

The Neuropsychological Battery comprises of several tests:

1. **Logical Memory** (Immediate and delayed recall)
2. **Rey Auditory Verbal Learning Test**
3. **Clock Drawing**
4. **Clock Copying**
5. **Fluency**
6. **Trail Making Test**
7. **Boston Naming Test**
8. **ANART (American National Adult Reading Test)**
9. **Digit Span**

Only a subset of these tests was administered at any given visit. The Digit Span was dropped after ADNI1. The Category Fluency - Animals was retained, but Vegetables were dropped after ADNI1. A Spanish language option for the Boston Naming Test became available in Spanish later in ADNI2/GO.


**Neuro-bat Scale:**  

****

Alzheimer's disease participants exhibit a score of ...

<u>Sources:<u>
- https://adni.bitbucket.io/reference/neurobat.html

In [289]:
nb_dict = pd.read_csv('../../data/tabular_data/NEUROBAT_dict.csv', header=None)
nb_dict

,0,1
0,Phase,Phase
1,ID,Record ID
2,RID,Participant roster ID
3,SITEID,Site ID
4,VISCODE,Visit code
...,...,...
76,RAVLT_forgetting,"""RAVLT Forgetting (trial 5 - delayed): Rey's ..."
77,RAVLT_immediate,"""AVLT (5 sum); Code -4"""
78,RAVLT_learning,"""RAVLT Learning (trial 5 - trial 1)"""
79,RAVLT_perc_forgetting,"""RAVLT Percent Forgetting"""


In [285]:
nb = pd.read_csv('../../data/tabular_data/NEUROBAT.csv')

# Change hidden missing values.
nb = nb.replace(-1, np.NaN)    
nb = nb.replace(-4, np.NaN)
nb = nb.replace('-1', np.NaN)    
nb = nb.replace('-4', np.NaN)
nb = nb.replace('-', np.NaN)    

pd.set_option("display.max_rows", 50, "display.max_columns", 50)

nb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14710 entries, 0 to 14709
Data columns (total 81 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Phase                  14710 non-null  object 
 1   ID                     14710 non-null  int64  
 2   RID                    14710 non-null  int64  
 3   SITEID                 14710 non-null  int64  
 4   VISCODE                14710 non-null  object 
 5   VISCODE2               14694 non-null  object 
 6   USERDATE               14710 non-null  object 
 7   USERDATE2              8452 non-null   object 
 8   EXAMDATE               5102 non-null   object 
 9   CLOCKCIRC              10783 non-null  float64
 10  CLOCKSYM               10783 non-null  float64
 11  CLOCKNUM               10783 non-null  float64
 12  CLOCKHAND              10783 non-null  float64
 13  CLOCKTIME              10785 non-null  float64
 14  CLOCKSCOR              10784 non-null  float64
 15  CO

C:\Users\kfatur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [286]:
nb_start_entries = nb.shape[0]

In [287]:
nb.isnull().sum()

Phase                        0
ID                           0
RID                          0
SITEID                       0
VISCODE                      0
VISCODE2                    16
USERDATE                     0
USERDATE2                 6258
EXAMDATE                  9608
CLOCKCIRC                 3927
CLOCKSYM                  3927
CLOCKNUM                  3927
CLOCKHAND                 3927
CLOCKTIME                 3925
CLOCKSCOR                 3926
COPYCIRC                  3921
COPYSYM                   3921
COPYNUM                   3921
COPYHAND                  3921
COPYTIME                  3922
COPYSCOR                  3919
LMSTORY                   8121
LIMMTOTAL                 4367
LIMMEND                   8131
AVTOT1                    3928
AVERR1                    3937
AVTOT2                    3934
AVERR2                    3941
AVTOT3                    3951
AVERR3                    3960
AVTOT4                    3963
AVERR4                    3973
AVTOT5  

In [ ]:

nb.isnull().sum()

In [ ]:
# How many entries did we lose to cleaning?
nb_finish_entries = nb.shape[0]
dropped_entries = nb_start_entries - nb_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/nb_start_entries*100, 2), '% removed')

In [ ]:
# Writing df to csv.
df = nb
df.to_csv('../../data/tabular_data/clean_NEUROBAT.csv', index=False)

###  6. Montreal Cognitive Assessment

The Montreal Cognitive Assessment (MoCA) is a rapid screening instrument to assess mild cognitive dysfunction. It covers different cognitive domains: 

1. **Attention and concentration**,
2. **Executive functions**, 
3. **Memory**, 
4. **Language**, 
5. **Visuoconstructional skills**, 
6. **Conceptual thinking**,
7. **Calculation**,
8. **Orientation**

**MoCa Scale (0-30):**  

**18-25 = mild cognitive impairment    
10-17 = moderate cognitive impairment   
< 10 = severe cognitive impairment**

"The cut-off score of 18 is usually considered to separate MCI from AD but there is overlap in the scores since, by definition, AD is determined by the presence of cognitive impairment in addition to loss of autonomy.   
The average MoCA score for MCI is 22 (range 19-25) and the average MoCA score for Mild AD is 16 (11-21)." (MoCa Test Website)

According to the ADNI Procedures Manual, the MOCA total score is not calculated for ADNI.

<u>Sources:<u>
- https://www.mocatest.org/faq/
- https://www.dementia.org/diagnosing-dementia-the-montreal-cognitive-assessment
    
<u>Test samples<u>:
- https://www.parkinsons.va.gov/resources/MOCA-Test-English.pdf
- https://de.scribd.com/document/393713047/Montreal-cognitive-assessment-Basic-English-FINAL-VERSION-4-June-2014-pdf

In [ ]:
nb_dict = pd.read_csv('../../data/tabular_data/NEUROBAT_dict.csv', header=None)
moca_dict

nb = pd.read_csv('../../data/tabular_data/NEUROBAT.csv', header=None)

# Change hidden missing values.
nb = nb.replace(-1, np.NaN)    
nb = nb.replace(-4, np.NaN)

nb.info()

nb_start_entries = nb.shape[0]

# Keep the columns we are interested in.
nb = nb[['ID', 'RID', 'MMSCORE', 'EXAMDATE']]

nb.isnull().sum()

In [ ]:
# How many entries did we lose to cleaning?
nb_finish_entries = mmse.shape[0]
dropped_entries = mmse_start_entries - mmse_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/mmse_start_entries*100, 2), '% removed')

In [ ]:
# How many entries did we lose to cleaning?
moca_finish_entries = moca.shape[0]
dropped_entries = moca_start_entries - moca_finish_entries
print(dropped_entries)
print(np.round(dropped_entries/moca_start_entries*100, 2), '% removed')

In [ ]:
# Writing df to csv.
df = moca
df.to_csv('../../data/tabular_data/clean_MOCA.csv', index=False)  

### <a id='MoCA'>7. Main Insights</a>